In [1]:
#Packages for connecting to ETrade
import configparser
import pandas as pd
import requests
import logging
import json
from rauth import OAuth1Service
from logging.handlers import RotatingFileHandler
import webbrowser
import datetime

#Additional packages for running trend system
import yfinance as yf
import math
import gc
from IPython.display import clear_output
import numpy as np
import time
import random

#Additional packages for system automation
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


In [2]:
#Excel file holding list of assets available to trade
input_file = r'C:\Users\reedx\OneDrive\Investing\Python\Code\Code packages\ETF list.xlsx'

#Path for saving trend trading system results
excel_export_path = r'C:\Users\reedx\OneDrive\Investing\Python\Code\Code packages\Output file.xlsx'

#Excel file for handling errors
error_file = r'C:\Users\reedx\OneDrive\Investing\Python\Code\Code packages\Error file.xlsx'
error_df = pd.read_excel(error_file)
error_df.set_index('date',inplace=True)


In [3]:

def get_ticker_prices(excel_file_path,tab_name,frequency):
    #Interval options: 1d, 5d, 1w, 1m, 3m
    
    #Get list of tickers
    ticker_df = pd.read_excel(excel_file_path, sheet_name=tab_name)
    
    global tickers
    tickers= ticker_df['Symbol']
    
    dfs = []

    for ticker in tickers:
        try:
            #Download OHLC data and organize into a dataframe
            data = yf.download(ticker, start=start_date, end=end_date,interval = frequency)
            data[f'{ticker} Open'] = data['Open']
            data[f'{ticker} High'] = data['High']        
            data[f'{ticker} Low'] = data['Low']        
            data[f'{ticker} Close'] = data['Close']
            data[f'{ticker} Adj Close'] = data['Adj Close']            
            data[f'{ticker} Volume'] = data['Volume']
            data = data[[f'{ticker} Open', f'{ticker} High', f'{ticker} Low', f'{ticker} Close', f'{ticker} Volume', f'{ticker} Adj Close']]
            
            dfs.append(data)
        except:
            pass

    df = pd.concat(dfs, axis=1)
    df.index = pd.to_datetime(df.index)
        
    return df



In [4]:
#Define trend system
def simple_trend(df,
                 ticker, 
                 start_date, #Start for analysis
                 end_date, #End for analysis
                 numerator_look_back, #short time period to test above denominator
                 denominator_look_back): #long time period to test below numerator
    
    chart_ticker = ticker
  
    chart_data = df[[f'{chart_ticker} Open',f'{chart_ticker} High',f'{chart_ticker} Low',f'{chart_ticker} Close',f'{chart_ticker} Volume',f'{chart_ticker} Adj Close']]
    chart_data = chart_data.rename(columns=lambda x: x.replace(f'{chart_ticker} ', ''))
    
    trend = chart_data['Close'].iloc[-numerator_look_back:].mean()/chart_data['Close'].iloc[-denominator_look_back:].mean() - 1
    last_price = chart_data['Close'].iloc[-1]
    
    trend_data = {'Ticker':chart_ticker,'Trend': trend,'Date':end_date,'Price':last_price}
    return trend_data

#Define function to pick top trends as desired positions
def simple_trend_positions(market, calendar_end_date):
    
    end_date = calendar_end_date
    start_date = calendar_end_date - datetime.timedelta(days=data_download_history)

    #For aggregating tickers
    global trends_df
    trends_df = pd.DataFrame(columns = ['Ticker','Trend','Date','Price'])

    #For storing position data
    global current_portfolio_df

    #Trim dataframe to new end_date
    df_trimmed = df[(df.index <= end_date)]
    
    #Run list of daily breakouts
    for ticker in tickers:
        try: 
            trend_output = simple_trend(df_trimmed,
                            ticker, 
                            start_date, #Start for analysis
                            end_date, #End for analysis
                            trend_numerator, #time period for numerator
                            trend_denominator) #time period for denominator
    
            trends_df.loc[len(trends_df)] = trend_output #.loc faster than concat for a new row      
        
        except:
            pass

    #Sort trends strongest to weakest, want to hold top 10
    top_trends_df = trends_df.sort_values(by='Trend', ascending=False).head(10)

    #Need to calculate cost of positions
    #Assume you buy at current closing price
    #If position is already held in the current_portfolio, then use that existing cost
    #New positions: Cost = current closing price
    #Existing positions: Retain existing cost

    #First assume cost = price (all new positions buy at current closing price)
    top_trends_df['Cost'] = top_trends_df['Price']

    #Then lookup cost if you already have that position in the current portfolio
    for idx, row in top_trends_df.iterrows():
        lookup_ticker = row['Ticker']
        try:
            cost = current_portfolio_df.loc[current_portfolio_df['Ticker'] == lookup_ticker, 'Cost'].values[-1]
            top_trends_df.loc[idx, 'Cost'] = cost
            
            date = current_portfolio_df.loc[current_portfolio_df['Ticker'] == lookup_ticker, 'Date'].values[-1]
            top_trends_df.loc[idx, 'Date'] = date
        except:
            pass
    
    #ADDING GAIN
    top_trends_df['Gain'] = top_trends_df['Price']/top_trends_df['Cost'] -1 #calculate gain
    #top_trends_df.loc[top_trends_df['Price'].isna(), 'Gain'] = 0 #Override gain calc if price is Nan
    #top_trends_df.loc[top_trends_df['Cost'].isna(), 'Gain'] = 0 #Override gain calc if cost is Nan

    #Calculating how long a position has been open, called 'days since transaction'
    top_trends_df['Days since transaction'] = 0

    #Using try so the first loop will convert the date, but date will already be converted for future loops
    try:
        current_portfolio_df['Date'] = pd.to_datetime(current_portfolio_df['Date'])
    except:
        pass

    #Loop to calculate days since transaction for each positions
    try:
        current_portfolio_df['Days since transaction'] = (calendar_end_date - current_portfolio_df['Date']).apply(lambda x: x.days)
        for idx, row in top_trends_df.iterrows():
            lookup_ticker = row['Ticker']
            try:
                days_since_transaction = current_portfolio_df.loc[current_portfolio_df['Ticker'] == lookup_ticker, 'Days since transaction'].values[-1]
                top_trends_df.loc[idx, 'Days since transaction'] = days_since_transaction
            except:
                pass
    except:
        pass

    #For estimating trade costs for each day
    new_tickers = set(top_trends_df['Ticker'].unique())  # Converted to set for easier comparison
    prior_tickers = set(current_portfolio_df['Ticker'].unique())

    #Find tickers that changed from the prior day (new positions)
    diff_tickers = new_tickers - prior_tickers  # Tickers in dataframe1 but not in dataframe2 

    #Count number of trades
    num_diff_tickers = len(diff_tickers)

    #Calculate an approximate portion of portfolio that is turning over each day
    #Multiply by 2 since both purchase and sale pays bid ask spread
    #times_paying_spread is 2 if all positions change, 1 if half of positions change, .25 if 1/8 of positions change, etc.
    #Later multiple times_paying_spread by approximate spread cost to gauge daily transaction costs
    global times_paying_spread
    times_paying_spread = num_diff_tickers/len(top_trends_df) * 2 

    #Update current portfolio to new positions
    #then function is looped over again the next day so current portfolio is positions from prior day
    current_portfolio_df = top_trends_df

    #save memory by deleting unused variables and clearing output
    gc.collect()
    #clear_output()
    
    return top_trends_df



In [5]:
#Get current positions

#Choose universe of assets
market = 'ETF universe'

#Set trend settings
calendar_end_date = datetime.date.today().strftime('%Y-%m-%d')
frequency = '1mo' #1d, 1wk, 1mo, 3mo
frequency_multiply_factor = 31 #1:day, 7: week, 31: month, 63:quarter (only used pull enough data to calculate trend)

trend_numerator = 1
trend_denominator = 10

periods_to_download = max(trend_numerator, trend_denominator) #Need to pull sufficient history to calculate trend on first date
data_download_history = periods_to_download*frequency_multiply_factor 

spread_cost = .1 #Approximate % bid-ask spread to calculate transaction costs (buy at the ask, sell at the bid)

#Convert dates
#Add 1 to end date (yfinance pulls data as of 1 day prior to specified end_date)
#Adjust start date to pull enough history to calculate trend for the first day in back test
end_date = datetime.datetime.strptime(calendar_end_date, "%Y-%m-%d").date() + datetime.timedelta(days=1) #USED FOR get_ticker_prices
start_date = end_date - datetime.timedelta(days=data_download_history) #start_date to get necessary history

df = get_ticker_prices(input_file,market,frequency)

#Set up dataframes
current_portfolio_df = pd.DataFrame(columns = ['Ticker','Date','Cost','Price'])

target_allocation_df = simple_trend_positions(market,pd.to_datetime(end_date)) #FIND POSITIONS

target_allocation_df = target_allocation_df[['Ticker']]
target_allocation_df['weight'] = 1/len(target_allocation_df)

print(target_allocation_df)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

$EWGS: possibly delisted; No price data found  (1mo 2024-03-21 -> 2025-01-25)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

    Ticker  weight
22    IBIT     0.1
93    BITQ     0.1
64    ARKK     0.1
71    ARKX     0.1
70    JETS     0.1
92    IBLC     0.1
136    EIS     0.1
116    XTL     0.1
38     XLY     0.1
91    CLOU     0.1


C:\Users\reedx\AppData\Local\Temp\ipykernel_7068\2377443578.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_allocation_df['weight'] = 1/len(target_allocation_df)


In [6]:
#Read consumer key and consumer secret from config file
config_path = r'C:\Users\reedx\OneDrive\Investing\Python\EtradePythonClient\etrade_python_client\config.ini'
config = configparser.ConfigParser()
config.read(config_path)

consumer_key = config['DEFAULT']['CONSUMER_KEY']
consumer_secret = config['DEFAULT']['CONSUMER_SECRET']


In [7]:
def auto_oauth(selection):
    
    max_attempts = 5

    #GOOGLE 'WHAT IS MY USER AGENT' TO UPDATE THE USER AGENT BELOW IF A BROWSER IS NOT LAUNCHING AS APPROPRIATE
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'    
    chrome_options = Options()
    chrome_options.add_argument(f"user-agent={user_agent}")
#    chrome_options.add_argument("--headless") #headless mode doesn't launch a browser window
    chrome_options.add_argument("--window-size=1920x1080") #set window size, recommended, idk why

    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)

    for attempt in range(1, max_attempts + 1):
        try:
            """Allows user authorization for the sample application with OAuth 1"""
            etrade = OAuth1Service(
                name="etrade",
                consumer_key=config["DEFAULT"]["CONSUMER_KEY"],
                consumer_secret=config["DEFAULT"]["CONSUMER_SECRET"],
                request_token_url="https://api.etrade.com/oauth/request_token",
                access_token_url="https://api.etrade.com/oauth/access_token",
                authorize_url="https://us.etrade.com/e/t/etws/authorize?key={}&token={}",
                base_url="https://api.etrade.com")

            if selection == "sandbox":
                base_url = config["DEFAULT"]["SANDBOX_BASE_URL"]
            elif selection == "live":
                base_url = config["DEFAULT"]["PROD_BASE_URL"]

            # Step 1: Get OAuth 1 request token and secret
            request_token, request_token_secret = etrade.get_request_token(
                params={"oauth_callback": "oob", "format": "json"})

            # Step 2: Go through the authentication flow. Login to E*TRADE.
            # After you login, the page will provide a verification code to enter.
            authorize_url = etrade.authorize_url.format(etrade.consumer_key, request_token)
            
            
            chromedriver_path = r'C:\Users\reedx\anaconda3\Lib\site-packages\chromedriver-win64\chromedriver'
            driver = webdriver.Chrome(options=chrome_options) #launching chrome
            time.sleep(3.1)
            driver.get(authorize_url)
    
            time.sleep(5.1)
            element = driver.find_element(By.ID, 'USER')
            time.sleep(1.2)
            element.send_keys(config["DEFAULT"]["login"]) 

            time.sleep(4.9)
            element = driver.find_element(By.ID, 'password')
            time.sleep(1.3)
            element.send_keys(config["DEFAULT"]["pw"])

            time.sleep(3.2)
            element = driver.find_element(By.ID, 'mfaLogonButton') #finding logon button
            time.sleep(.5)
            element.click() #clicking logon button

            time.sleep(5.8)
            element = driver.find_element(By.CSS_SELECTOR,"input[type='submit'][value='Accept']") #accepting terms and conditions
            time.sleep(.3)
            element.click() #clicking terms and conditions

            time.sleep(2.7)    
            element = driver.find_element(By.XPATH, '//div[@style="text-align:center"]/input[@type="text"]') #finding the text code
            text_code = element.get_attribute('value').strip() #copying the text code

            # Step 3: Exchange the authorized request token for an authenticated OAuth 1 session

            session = etrade.get_auth_session(request_token,
                                          request_token_secret,
                                          params={"oauth_verifier": text_code}) #starting authorized session

            print("Successful authorization")

            driver.quit() #quitting driver session to save compute power
            
            return session, base_url
        
            break
        except Exception as e: #if there is an error then continue to run through the loop again
            time.sleep(5)
            
    else:
        
        print("Unable to login")
        
        dummary_var = 1
        dummary_var2 = 1
    
        return dummary_var, dummary_var2


In [8]:
#Start authorized session
try:
    selection = "live" #set to either "sandbox" or "live"
    session, base_url = auto_oauth(selection) #start authorized session

except:
    error = pd.DataFrame({'date':[end_date.strftime('%Y-%m-%d')], 'error': ['Failed with authentication']})
    error.set_index('date',inplace=True)
    error_combined = pd.concat([error_df,error], ignore_index = False)
    
    writer = pd.ExcelWriter(error_file)
    error_combined.to_excel(writer, index=True)
    writer.close()
    raise ValueError('Error')

Successful authorization


In [18]:
#Set up logger settings
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)
handler = RotatingFileHandler("python_client.log", maxBytes=5 * 1024 * 1024, backupCount=3)
FORMAT = "%(asctime)-15s %(message)s"
fmt = logging.Formatter(FORMAT, datefmt='%m/%d/%Y %I:%M:%S %p')
handler.setFormatter(fmt)
logger.addHandler(handler)

In [10]:
class My_Accounts:

    def __init__(self, session, base_url):
        #Initialize Accounts object with session and account information
        #param session: authenticated session

        self.session = session
        self.account = {}
        self.base_url = base_url

    def get_account_details(self,account_index):
        
        #Calls account list API to retrieve a list of the user's E*TRADE accounts
        #param self:Passes in parameter authenticated session

        # URL for the API endpoint
        url = self.base_url + "/v1/accounts/list.json"

        # Make API call for GET request
        response = self.session.get(url, header_auth=True)
        logger.debug("Request Header: %s", response.request.headers)

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()
            
            if data is not None and "AccountListResponse" in data and "Accounts" in data["AccountListResponse"] \
                    and "Account" in data["AccountListResponse"]["Accounts"]:
                accounts = data["AccountListResponse"]["Accounts"]["Account"]

                accounts[:] = [d for d in accounts if d.get('accountStatus') != 'CLOSED']
                account_ID = accounts[account_index]['accountId']
                account_ID_key = accounts[account_index]['accountIdKey']
                institution_type = accounts[account_index]['institutionType']

            else:
                # Handle errors
                logger.debug("Response Body: %s", response.text)
                if response is not None and response.headers['Content-Type'] == 'application/json' \
                        and "Error" in response.json() and "message" in response.json()["Error"] \
                        and response.json()["Error"]["message"] is not None:
                    print("Error: " + data["Error"]["message"])
                else:
                    print("Error: AccountList API service error")
        else:
            # Handle errors
            logger.debug("Response Body: %s", response.text)
            if response is not None and response.headers['Content-Type'] == 'application/json' \
                    and "Error" in response.json() and "message" in response.json()["Error"] \
                    and response.json()["Error"]["message"] is not None:
                print("Error: " + response.json()["Error"]["message"])
            else:
                print("Error: AccountList API service error")
        return account_ID, account_ID_key, institution_type

    def balance(self, account_ID_key,institution_type):

        #Calls account balance API to retrieve the current balance and related details for a specified account
        #param self: Pass in parameters authenticated session and information on selected account

        # URL for the API endpoint
        url = self.base_url + "/v1/accounts/" + account_ID_key + "/balance.json"

        # Add parameters and header information
        params = {"instType": institution_type, "realTimeNAV": "true"}
        headers = {"consumerkey": consumer_key}

        # Make API call for GET request
        response = self.session.get(url, header_auth=True, params=params, headers=headers)
        logger.debug("Request url: %s", url)
        logger.debug("Request Header: %s", response.request.headers)

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()
            if data is not None and "BalanceResponse" in data:
                balance_data = data["BalanceResponse"]
                if balance_data is not None and "accountId" in balance_data:
                    print("\n\nBalance for " + balance_data["accountId"] + ":")
                else:
                    print("\n\nBalance:")
                # Display balance information
                if balance_data is not None and "accountDescription" in balance_data:
                    print("Account Nickname: " + balance_data["accountDescription"])
                if balance_data is not None and "Computed" in balance_data \
                        and "RealTimeValues" in balance_data["Computed"] \
                        and "totalAccountValue" in balance_data["Computed"]["RealTimeValues"]:
                    print("Net Account Value: "
                          + str('${:,.2f}'.format(balance_data["Computed"]["RealTimeValues"]["totalAccountValue"])))
                if balance_data is not None and "Computed" in balance_data \
                        and "marginBuyingPower" in balance_data["Computed"]:
                    print("Margin Buying Power: " + str('${:,.2f}'.format(balance_data["Computed"]["marginBuyingPower"])))
                if balance_data is not None and "Computed" in balance_data \
                        and "cashBuyingPower" in balance_data["Computed"]:
                    print("Cash Buying Power: " + str('${:,.2f}'.format(balance_data["Computed"]["cashBuyingPower"])))
            else:
                # Handle errors
                logger.debug("Response Body: %s", response.text)
                if response is not None and response.headers['Content-Type'] == 'application/json' \
                        and "Error" in response.json() and "message" in response.json()["Error"] \
                        and response.json()["Error"]["message"] is not None:
                    print("Error: " + response.json()["Error"]["message"])
                else:
                    print("Error: Balance API service error")
        else:
            # Handle errors
            logger.debug("Response Body: %s", response.text)
            if response is not None and response.headers['Content-Type'] == 'application/json' \
                    and "Error" in response.json() and "message" in response.json()["Error"] \
                    and response.json()["Error"]["message"] is not None:
                print("Error: " + response.json()["Error"]["message"])
            else:
                print("Error: Balance API service error")

    def portfolio(self,account_ID_key):
        
        #Call portfolio API to retrieve a list of positions held in the specified account (pulls 50 transactions at a time, 2nd loop handles 51+)
        #param self: Passes in parameter authenticated session and information on selected account
        

        # URL for the API endpoint
        url =self.base_url + "/v1/accounts/" + account_ID_key + "/portfolio.json"

        # Make API call for GET request
        response = self.session.get(url, header_auth=True)
        logger.debug("Request Header: %s", response.request.headers)

        print("\nPortfolio:")

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()

            if data is not None and "PortfolioResponse" in data and "AccountPortfolio" in data["PortfolioResponse"]:
                # Display balance information
                for acctPortfolio in data["PortfolioResponse"]["AccountPortfolio"]:
                    if acctPortfolio is not None and "Position" in acctPortfolio:
                        for position in acctPortfolio["Position"]:
                            print_str = ""
                            if position is not None and "symbolDescription" in position:
                                print_str = print_str + "Symbol: " + str(position["symbolDescription"])
                            if position is not None and "quantity" in position:
                                print_str = print_str + " | " + "Quantity #: " + str(position["quantity"])
                            if position is not None and "Quick" in position and "lastTrade" in position["Quick"]:
                                print_str = print_str + " | " + "Last Price: " \
                                            + str('${:,.2f}'.format(position["Quick"]["lastTrade"]))
                            if position is not None and "pricePaid" in position:
                                print_str = print_str + " | " + "Price Paid $: " \
                                            + str('${:,.2f}'.format(position["pricePaid"]))
                            if position is not None and "totalGain" in position:
                                print_str = print_str + " | " + "Total Gain $: " \
                                            + str('${:,.2f}'.format(position["totalGain"]))
                            if position is not None and "marketValue" in position:
                                print_str = print_str + " | " + "Value $: " \
                                            + str('${:,.2f}'.format(position["marketValue"]))
                            print(print_str)
                    else:
                        print("None")
            else:
                # Handle errors
                logger.debug("Response Body: %s", response.text)
                if response is not None and "headers" in response and "Content-Type" in response.headers \
                        and response.headers['Content-Type'] == 'application/json' \
                        and "Error" in response.json() and "message" in response.json()["Error"] \
                        and response.json()["Error"]["message"] is not None:
                    print("Error: " + response.json()["Error"]["message"])
                else:
                    print("Error: Portfolio API service error")
        elif response is not None and response.status_code == 204:
            print("None")
        else:
            # Handle errors
            logger.debug("Response Body: %s", response.text)
            if response is not None and "headers" in response and "Content-Type" in response.headers \
                    and response.headers['Content-Type'] == 'application/json' \
                    and "Error" in response.json() and "message" in response.json()["Error"] \
                    and response.json()["Error"]["message"] is not None:
                print("Error: " + response.json()["Error"]["message"])
            else:
                print("Error: Portfolio API service error")
                
    def get_past_transactions(self,account_ID_key,start,end,buy_sell):
        
        #Call transactions API to retrieve a list of transactions
        #Organize purchases over last 30 days in a dataframe
        #param self: Passes in parameter authenticated session and information on selected account
                
        url = self.base_url + "/v1/accounts/" + account_ID_key + "/transactions.json"

        headers = {"consumerkey": consumer_key}
        parameters = {"startDate":start,"endDate":end,"count":50}

        response_executed = session.get(url, header_auth=True, params = parameters, headers=headers)

        logger.debug("Request Header: %s", response_executed.request.headers)
        logger.debug("Response Body: %s", response_executed.text)
        logger.debug(response_executed.text)

        # Handle and parse response
        if response_executed.status_code == 204:
            logger.debug(response_executed)
            print("None")
        elif response_executed.status_code == 200:
            parsed = json.loads(response_executed.text)
            logger.debug(json.dumps(parsed, indent=4, sort_keys=True))

            purchases_30d = pd.DataFrame(columns=['Ticker','Cost','Date'])

            data = response_executed.json()

            transactions = data["TransactionListResponse"]["Transaction"]
            transaction_count = data['TransactionListResponse']['transactionCount']
            
            try:
                marker_value = data['TransactionListResponse']['marker'] #marker used to pull next set of transactions
            except:
                marker_value = 'No marker, this is the last pull'
            
            global marker
            marker = marker_value         
            
            transactions[:] = [d for d in transactions if d.get('transactionType') == buy_sell]

            for i in range(len(transactions)):
                ticker = transactions[i-1]['brokerage']['product']['symbol']
                cost = transactions[i-1]['brokerage']['price']
                date = transactions[i-1]['transactionDate']
                date = datetime.datetime.fromtimestamp(date/1000)
                new_row = pd.DataFrame({'Ticker':[ticker], 'Cost':[cost],'Date':[date]})

                purchases_30d = pd.concat([purchases_30d,new_row], ignore_index = True)
                
            #Another loop to pull transactions again
            #Pulls transactions 51+ (can only pull 50 transactions at a time)
            #Changes to pull those transactions:
            #Adding 'marker':marker in the api pull
            #Take existing purchases_30d from prior loop and adding rows
            while transaction_count == 50:
                url = self.base_url + "/v1/accounts/" + account_ID_key + "/transactions.json"

                headers = {"consumerkey": consumer_key}
                parameters = {"startDate":start,"endDate":end,"count":50,'marker':marker}

                response_executed = session.get(url, header_auth=True, params = parameters, headers=headers)

                print(response_executed)

                logger.debug("Request Header: %s", response_executed.request.headers)
                logger.debug("Response Body: %s", response_executed.text)
                logger.debug(response_executed.text)

                print("\nTransactions last 30d:")
                # Handle and parse response
                if response_executed.status_code == 204:
                    logger.debug(response_executed)
                    print("None")
                elif response_executed.status_code == 200:
                    parsed = json.loads(response_executed.text)
                    logger.debug(json.dumps(parsed, indent=4, sort_keys=True))
                    data = response_executed.json()

                    transactions = data["TransactionListResponse"]["Transaction"]
                    
                    try:
                        marker_value = data['TransactionListResponse']['marker'] #marker used to pull next set of transactions
                    except:
                        marker_value = 'No marker, this is the last pull'
                        
                    transaction_count = data['TransactionListResponse']['transactionCount']
                    marker = marker_value

                    transactions[:] = [d for d in transactions if d.get('transactionType') == buy_sell]

                    for i in range(len(transactions)):
                        ticker = transactions[i-1]['brokerage']['product']['symbol']
                        cost = transactions[i-1]['brokerage']['price']
                        date = transactions[i-1]['transactionDate']
                        date = datetime.datetime.fromtimestamp(date/1000)
                        new_row = pd.DataFrame({'Ticker':[ticker], 'Cost':[cost],'Date':[date]})
                        purchases_30d = pd.concat([purchases_30d,new_row], ignore_index = True)

            return purchases_30d                     

    def portfolio(self,account_ID_key):
        
        #Call portfolio API to retrieve a list of positions held in the specified account
        #param self: Passes in parameter authenticated session and information on selected account

        # URL for the API endpoint
        url =self.base_url + "/v1/accounts/" + account_ID_key + "/portfolio.json"

        # Make API call for GET request
        response = self.session.get(url, header_auth=True)
        logger.debug("Request Header: %s", response.request.headers)

        #Handle response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()

            if data is not None and "PortfolioResponse" in data and "AccountPortfolio" in data["PortfolioResponse"]:

                current_portfolio_df = pd.DataFrame(columns=['Ticker','Quantity','Last Price','Total Value'])

                # Display balance information
                for acctPortfolio in data["PortfolioResponse"]["AccountPortfolio"]:
                    if acctPortfolio is not None and "Position" in acctPortfolio:
                        for position in acctPortfolio["Position"]:
                            if position is not None and "symbolDescription" in position:
                                ticker = str(position['symbolDescription'])
                            if position is not None and "quantity" in position:
                                quantity = position['quantity']
                            if position is not None and "Quick" in position and "lastTrade" in position["Quick"]:
                                last_price = position['Quick']['lastTrade']
                            if position is not None and "marketValue" in position:
                                total_value = position['marketValue']
                            #can also add 'pricePaid' and 'totalGain' from position data

                            new_row = pd.DataFrame({'Ticker':[ticker], 'Quantity':[quantity],'Last Price':[last_price], 'Total Value':[total_value]})
                            current_portfolio_df = pd.concat([current_portfolio_df,new_row], ignore_index = True)
                            
                    else:
                        print("None")
            else:
                # Handle errors
                logger.debug("Response Body: %s", response.text)
                if response is not None and "headers" in response and "Content-Type" in response.headers \
                        and response.headers['Content-Type'] == 'application/json' \
                        and "Error" in response.json() and "message" in response.json()["Error"] \
                        and response.json()["Error"]["message"] is not None:
                    print("Error: " + response.json()["Error"]["message"])
                else:
                    print("Error: Portfolio API service error")
        elif response is not None and response.status_code == 204:
            print("None")
        else:
            # Handle errors
            logger.debug("Response Body: %s", response.text)
            if response is not None and "headers" in response and "Content-Type" in response.headers \
                    and response.headers['Content-Type'] == 'application/json' \
                    and "Error" in response.json() and "message" in response.json()["Error"] \
                    and response.json()["Error"]["message"] is not None:
                print("Error: " + response.json()["Error"]["message"])
            else:
                print("Error: Portfolio API service error")
        return current_portfolio_df
    
    def get_balance(self, account_ID_key,institution_type):
        
        #Calls account balance API to retrieve the current balance and related details for a specified account
        #param self: Pass in parameters authenticated session and information on selected account

        # URL for the API endpoint
        url = self.base_url + "/v1/accounts/" + account_ID_key + "/balance.json"

        # Add parameters and header information
        params = {"instType": institution_type, "realTimeNAV": "true"}
        headers = {"consumerkey": consumer_key}

        # Make API call for GET request
        response = self.session.get(url, header_auth=True, params=params, headers=headers)
        logger.debug("Request url: %s", url)
        logger.debug("Request Header: %s", response.request.headers)

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()
            if data is not None and "BalanceResponse" in data:
                balance_data = data["BalanceResponse"]
                # Display balance information
                if balance_data is not None and "Computed" in balance_data \
                        and "RealTimeValues" in balance_data["Computed"] \
                        and "totalAccountValue" in balance_data["Computed"]["RealTimeValues"]:
                    print("Net Account Value: " + str('${:,.2f}'.format(balance_data["Computed"]["RealTimeValues"]["totalAccountValue"])))
                    account_balance = balance_data["Computed"]["RealTimeValues"]["totalAccountValue"]

                if balance_data is not None and "Computed" in balance_data \
                        and "cashBuyingPower" in balance_data["Computed"]:
                    print("Cash Buying Power: " + str('${:,.2f}'.format(balance_data["Computed"]["cashBuyingPower"])))
                    account_cash_buying_power = balance_data["Computed"]["cashBuyingPower"]
            else:
                # Handle errors
                logger.debug("Response Body: %s", response.text)
                if response is not None and response.headers['Content-Type'] == 'application/json' \
                        and "Error" in response.json() and "message" in response.json()["Error"] \
                        and response.json()["Error"]["message"] is not None:
                    print("Error: " + response.json()["Error"]["message"])
                else:
                    print("Error: Balance API service error")
        else:
            # Handle errors
            logger.debug("Response Body: %s", response.text)
            if response is not None and response.headers['Content-Type'] == 'application/json' \
                    and "Error" in response.json() and "message" in response.json()["Error"] \
                    and response.json()["Error"]["message"] is not None:
                print("Error: " + response.json()["Error"]["message"])
            else:
                print("Error: Balance API service error")
        return account_balance, account_cash_buying_power

    def get_quote(self,symbols):
        
        #Calls account balance API to retrieve the current balance and related details for a specified account
        #param self: Pass in parameters authenticated session and information on selected account
        
        quote_df = pd.DataFrame(columns = ['Ticker','ask','bid','last_trade','last_trade_time','last_trade_minutes_ago'])
        
        # URL for the API endpoint
        url = self.base_url + "/v1/market/quote/" + symbols + ".json"

        # Make API call for GET request
        response = self.session.get(url, header_auth=True)
        
        logger.debug("Request url: %s", url)
        logger.debug("Request Header: %s", response.request.headers)

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()
            
            if data is not None and "QuoteResponse" in data:
                quote_response = data["QuoteResponse"]
                # Display balance information
                if quote_response is not None and "QuoteData" in quote_response:
                    if quote_response is not None and "QuoteData" in quote_response:
                        
                        quote_data = quote_response["QuoteData"] #this is where you can iterate for multiple quotes
                        for quote in quote_data:
                            ticker = quote['Product']['symbol']
                            ask = quote['All']['ask']
                            bid = quote['All']['bid']
                            last_trade = quote['All']['lastTrade']
                            last_trade_time = quote['All']['timeOfLastTrade']
                            last_trade_time = datetime.datetime.fromtimestamp(last_trade_time)
                            last_trade_minutes_ago = (datetime.datetime.now() - last_trade_time).total_seconds()/60
                            
                            new_row = {'Ticker':ticker,'ask':ask,'bid':bid,
                                       'last_trade':last_trade,'last_trade_time':last_trade_time,'last_trade_minutes_ago':last_trade_minutes_ago}
                            new_row = pd.DataFrame([new_row])
                            quote_df = pd.concat([quote_df,new_row],ignore_index=True)
                            
            else:
                # Handle errors
                logger.debug("Response Body: %s", response.text)
                if response is not None and response.headers['Content-Type'] == 'application/json' \
                        and "Error" in response.json() and "message" in response.json()["Error"] \
                        and response.json()["Error"]["message"] is not None:
                    print("Error: " + response.json()["Error"]["message"])
                else:
                    print("Error: Balance API service error")
        else:
            # Handle errors
            logger.debug("Response Body: %s", response.text)
            if response is not None and response.headers['Content-Type'] == 'application/json' \
                    and "Error" in response.json() and "message" in response.json()["Error"] \
                    and response.json()["Error"]["message"] is not None:
                print("Error: " + response.json()["Error"]["message"])
            else:
                print("Error: Balance API service error")
        return quote_df

In [11]:
#1. Get account ID
#2. Use account ID to get account balance
#3. Get existing positions


accounts = My_Accounts(session, base_url)

#Get account ID (used for all other account details)
account_ID, account_ID_key, institution_type = accounts.get_account_details(0) #Entering 0 here to get the ID for the 1st account

#Get account balance and cash balance
account_balance, account_cash_buying_power = accounts.get_balance(account_ID_key, institution_type)

#Get current portfolio              
current_portfolio_df = accounts.portfolio(account_ID_key)

current_portfolio_df


Net Account Value: $10,861.40
Cash Buying Power: $28.63


C:\Users\reedx\AppData\Local\Temp\ipykernel_7068\3192054314.py:327: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_portfolio_df = pd.concat([current_portfolio_df,new_row], ignore_index = True)


,Ticker,Quantity,Last Price,Total Value
0,DJP,18,33.91,610.3799
1,XLU,7,78.69,550.8300
2,XLP,14,77.50,1085.0000
3,XLE,5,92.20,461.0000
4,TIP,40,107.24,4289.6000
5,SPLV,11,70.82,779.0199
6,QQQ,1,532.64,532.6399
7,GOVT,74,22.43,1659.8199
8,XLV,6,144.08,864.4800


In [12]:
def calc_shares_to_trade_rounded(transaction_df):
    
    #finding transaction values after rounding shares to nearest integer (no fractional sales, sell orders have negative values)
    
    transaction_df['transaction_value_rounded'] = np.where(transaction_df['order'] == 'Buy', transaction_df['last_trade'] * transaction_df['shares_to_trade_rounded'], 
                                                           np.where(transaction_df['order'] == 'Sell', -transaction_df['last_trade'] * transaction_df['shares_to_trade_rounded'],0))

    transaction_df['transaction_value_rounding_error'] = transaction_df['transaction_value_rounded'] - transaction_df['Transaction Value']
    transaction_df['transaction_value_rounding_error'] = round(transaction_df['transaction_value_rounding_error'],2)

    transaction_df['buy_rounding_error'] = np.where(transaction_df['order'] == 'Buy', transaction_df['transaction_value_rounding_error'], np.nan)
    transaction_df['sell_rounding_error'] = np.where(transaction_df['order'] == 'Sell', transaction_df['transaction_value_rounding_error'], np.nan)
    
    return transaction_df

def find_transactions():
    
    #Find buy and sell orders needed to get current portfolio to target portfolio (target_allocation_df)
    
    transaction_df = pd.merge(target_allocation_df,current_portfolio_df, how='outer')
    
    #Create list of tickers for current and target positions
    #Use get_quote function to pull bid/ask data
    ticker_list = transaction_df['Ticker'].tolist()
    ticker_string = ','.join(ticker_list)

    accounts = My_Accounts(session, base_url)
    quote_df = accounts.get_quote(ticker_string)
        
    transaction_df = pd.merge(transaction_df,quote_df[['Ticker','last_trade','ask','bid']],left_on='Ticker',right_on='Ticker',how='left')
    
    #Calculate target dollar value of each position ['Target Value']
    #Calculate upper and lower bounds for that position
    #Rebalance if position is 33% larger or smaller than target (position is closed or has a dramatic rally for profit taking)
    transaction_df['weight'] = transaction_df['weight'].fillna(0) #Fill NaNs as 0
    transaction_df['Target Value'] = transaction_df['weight']*(account_balance) #Target dollar value of each position
    transaction_df['weight_lower_bound'] = transaction_df['weight'] - (transaction_df['weight']*.33)
    transaction_df['weight_upper_bound'] = transaction_df['weight'] + (transaction_df['weight']*.33)
    
    #Calculate current % weight in each position
    transaction_df['Total Value'] = transaction_df['Total Value'].fillna(0)
    transaction_df['current_weight'] = transaction_df['Total Value']/(account_balance)
    
    #Calculate total transaction value if current weight is outside upper or lower bound
    transaction_df['Transaction Value'] = np.where(transaction_df['current_weight'] > transaction_df['weight_upper_bound'],
                                                   transaction_df['Target Value'] - transaction_df['Total Value'],
                                                   np.where(transaction_df['current_weight'] < transaction_df['weight_lower_bound'],
                                                           transaction_df['Target Value'] - transaction_df['Total Value'],
                                                           0))
    
    #Find number of shares to trade (starting point) and buy/sell
    transaction_df['shares_to_trade'] = abs(transaction_df['Transaction Value']/transaction_df['last_trade'])
    transaction_df['shares_to_trade'] = transaction_df['shares_to_trade'] + .001 #adding .001 so math.floor at an even integer doesn't round down to next integer below it
    transaction_df['order'] = np.where(transaction_df['Transaction Value'] > 0, 'Buy', np.where(transaction_df['Transaction Value'] < 0, 'Sell','No trade'))
    
    #Rund to nearest integer for shares_to_trade_rounded
    #Then ensure that you are not selling more than you owe (if order == 'Sell' then min(shares_to_trade_rounded,quantity))
    transaction_df['rounding_shares_to_trade'] = round(transaction_df['shares_to_trade'])    
    transaction_df['shares_to_trade_rounded'] = np.where(
        (transaction_df['order'] == 'Sell') & 
        (transaction_df['Quantity'] < transaction_df['rounding_shares_to_trade']),
        transaction_df['Quantity'],
        transaction_df['rounding_shares_to_trade'])    
    transaction_df['shares_to_trade_rounded'] = transaction_df['shares_to_trade_rounded'].astype(int) #Convert to integers    

    #Run function that calculates the total value of trades (after rounding to nearest integer)
    transaction_df = calc_shares_to_trade_rounded(transaction_df)
    trade_sum_rounded = transaction_df['transaction_value_rounded'].sum()

    #Need to reduce buy orders until the transaction value of purchases <= value of sales + account cash
    #Only can trim buy orders that are > 0 (can't buy 0 or negative shares)
    #Trim buy orders on dataframe called _trim_options, then copy those buy orders to original transaction_df
    transaction_df_trim_options = transaction_df[transaction_df['shares_to_trade_rounded'] > 0]

    #Loops to find buy order (rounded) that has the largest difference from desired buy size (before rounding)
    if len(transaction_df_trim_options) > 0:
        
        #Identify row and value for largest buy error (difference in value in rounded trade vs. desired trade size before rounding)
        row_with_largest_diff = transaction_df_trim_options['buy_rounding_error'].idxmax()
        largest_buy_error = transaction_df_trim_options['buy_rounding_error'].max()

        #Continue to reduce buy order size by 1 share at a time until transaction value of buys <= value of sales + account cash
        while trade_sum_rounded > account_cash_buying_power:

            #Subtract 1 share from buy order that has largest buy rounding error
            transaction_df_trim_options.at[row_with_largest_diff, 'shares_to_trade_rounded'] -= 1

            #Rewrite transaction_df shares_to_trade_rounded from transaction_df_trim_options (to reflect reduced buy order size in main transaction_df)             
            transaction_df.loc[transaction_df['Ticker'].isin(transaction_df_trim_options['Ticker']), 'shares_to_trade_rounded'] = transaction_df_trim_options.loc[transaction_df_trim_options['Ticker'].isin(transaction_df['Ticker']), 'shares_to_trade_rounded'].values

            #Then run calc_shares_to_trade rounded again on transaction_df to find total value of rounded trades        
            calc_shares_to_trade_rounded(transaction_df)
            trade_sum_rounded = transaction_df['transaction_value_rounded'].sum()

            #Only can trim buy orders that are > 0 (can't buy 0 or negative shares)
            #So trim buy orders on dataframe called _trim_options, then copy those buy orders to original transaction_df
            transaction_df_trim_options = transaction_df[transaction_df['shares_to_trade_rounded'] > 0]
            row_with_largest_diff = transaction_df_trim_options['buy_rounding_error'].idxmax()
            largest_buy_error = transaction_df_trim_options['buy_rounding_error'].max()
            
            #while will continue to loop until the total trade value of buys < sales + cash 

        #Simplify transaction_df to only buys and sales (remove all rows where shares to trade = 0)
        transaction_df = transaction_df[transaction_df['shares_to_trade_rounded'] != 0]
        transaction_df['ending weight estimate'] = (transaction_df['transaction_value_rounded'] +  transaction_df['Total Value'])/(account_balance)
        transaction_df = transaction_df[['Ticker','Quantity','current_weight','weight','ending weight estimate','weight_lower_bound','weight_upper_bound','last_trade','bid','ask','order','shares_to_trade_rounded','Transaction Value','transaction_value_rounded']]       
        trade_sum_rounded = transaction_df['transaction_value_rounded'].sum()
    
    return transaction_df

In [13]:
transaction_df = find_transactions()
transaction_df

C:\Users\reedx\AppData\Local\Temp\ipykernel_7068\3192054314.py:451: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  quote_df = pd.concat([quote_df,new_row],ignore_index=True)


,Ticker,Quantity,current_weight,weight,ending weight estimate,weight_lower_bound,weight_upper_bound,last_trade,bid,ask,order,shares_to_trade_rounded,Transaction Value,transaction_value_rounded
0,IBIT,NaN,0.000000,0.1,9.742943e-02,0.067,0.133,58.79,59.99,60.00,Buy,18,1086.1400,1058.22
1,BITQ,NaN,0.000000,0.1,9.990885e-02,0.067,0.133,19.73,19.73,22.79,Buy,55,1086.1400,1085.15
2,ARKK,NaN,0.000000,0.1,9.812087e-02,0.067,0.133,62.69,62.96,63.00,Buy,17,1086.1400,1065.73
3,ARKX,NaN,0.000000,0.1,1.004373e-01,0.067,0.133,21.39,20.53,21.80,Buy,51,1086.1400,1090.89
4,JETS,NaN,0.000000,0.1,9.916493e-02,0.067,0.133,26.27,23.69,26.71,Buy,41,1086.1400,1077.07
5,IBLC,NaN,0.000000,0.1,1.001786e-01,0.067,0.133,38.86,30.46,46.58,Buy,28,1086.1400,1088.08
6,EIS,NaN,0.000000,0.1,9.740365e-02,0.067,0.133,81.38,72.47,93.14,Buy,13,1086.1400,1057.94
7,XTL,NaN,0.000000,0.1,1.031819e-01,0.067,0.133,112.07,96.65,128.57,Buy,10,1086.1400,1120.70
8,XLY,NaN,0.000000,0.1,8.522658e-02,0.067,0.133,231.42,231.02,231.96,Buy,4,1086.1400,925.68
9,CLOU,NaN,0.000000,0.1,1.006684e-01,0.067,0.133,24.85,25.01,25.19,Buy,44,1086.1400,1093.40


In [14]:
#Functions to excecute market orders

class Order:

    def __init__(self, session, account_ID_key, base_url):
        self.session = session
        self.base_url = base_url 
    
    def preview_and_place_market_order(self,symbol,order_action,quantity):
        
        #Function to structure market order data
        #Then excecutes function that places market order (next function)

        order = {"price_type": "MARKET",
                 "order_term": "GOOD_FOR_DAY",
                 "symbol": symbol,
                 "order_action": order_action,
                 "limit_price":None,
                 "quantity": quantity}
        
        #Create random unique order ID
        order["client_order_id"] = random.randint(1000000000, 9999999999)

        # URL for the API endpoint
        url = self.base_url + "/v1/accounts/" + account_ID_key + "/orders/preview.json"

        # Add parameters and header information
        headers = {"Content-Type": "application/xml", "consumerKey": config["DEFAULT"]["CONSUMER_KEY"]}

        # Add payload for POST Request
        payload = """<PreviewOrderRequest>
                       <orderType>EQ</orderType>
                       <clientOrderId>{0}</clientOrderId>
                       <Order>
                           <allOrNone>false</allOrNone>
                           <priceType>{1}</priceType>
                           <orderTerm>{2}</orderTerm>
                           <marketSession>REGULAR</marketSession>
                           <stopPrice></stopPrice>
                           <limitPrice>{3}</limitPrice>
                           <Instrument>
                               <Product>
                                   <securityType>EQ</securityType>
                                   <symbol>{4}</symbol>
                               </Product>
                               <orderAction>{5}</orderAction>
                               <quantityType>QUANTITY</quantityType>
                               <quantity>{6}</quantity>
                           </Instrument>
                       </Order>
                   </PreviewOrderRequest>"""

        #Structure data to pass to API
        payload = payload.format(order["client_order_id"], 
                                 order["price_type"], order["order_term"],
                                 order["limit_price"], order["symbol"], order["order_action"], order["quantity"])

        # Make API call for POST request
        response = self.session.post(url, header_auth=True, headers=headers, data=payload)
        logger.debug("Request Header: %s", response.request.headers)
        logger.debug("Request payload: %s", payload)

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()
            print("\nPreview Order:")

            if data is not None and "PreviewOrderResponse" in data and "PreviewIds" in data["PreviewOrderResponse"]:
                for previewids in data["PreviewOrderResponse"]["PreviewIds"]:
                    print("Preview ID: " + str(previewids["previewId"]))
                    preview_id = previewids['previewId']
                    self.place_market_order(order,preview_id) #RUNNING ORDER EXECUTION FUNCTION HERE
            else:
                # Handle errors
                data = response.json()
                if 'Error' in data and 'message' in data["Error"] and data["Error"]["message"] is not None:
                    print("Error: " + data["Error"]["message"])
                else:
                    print("Error: Preview Order API service error")

            if data is not None and "PreviewOrderResponse" in data and "Order" in data["PreviewOrderResponse"]:
                for orders in data["PreviewOrderResponse"]["Order"]:
                    order["limitPrice"] = orders["limitPrice"]

                    if orders is not None and "Instrument" in orders:
                        for instrument in orders["Instrument"]:
                            if instrument is not None and "orderAction" in instrument:
                                print("Action: " + instrument["orderAction"])
                            if instrument is not None and "quantity" in instrument:
                                print("Quantity: " + str(instrument["quantity"]))
                            if instrument is not None and "Product" in instrument \
                                    and "symbol" in instrument["Product"]:
                                print("Symbol: " + instrument["Product"]["symbol"])
                            if instrument is not None and "symbolDescription" in instrument:
                                print("Description: " + str(instrument["symbolDescription"]))

                if orders is not None and "priceType" in orders and "limitPrice" in orders:
                    print("Price Type: " + orders["priceType"])
                    if orders["priceType"] == "MARKET":
                        print("Price: MKT")
                    else:
                        print("Price: " + str(orders["limitPrice"]))
                if orders is not None and "orderTerm" in orders:
                    print("Duration: " + orders["orderTerm"])
                if orders is not None and "estimatedCommission" in orders:
                    print("Estimated Commission: " + str(orders["estimatedCommission"]))
                if orders is not None and "estimatedTotalAmount" in orders:
                    print("Estimated Total Cost: " + str(orders["estimatedTotalAmount"]))
            else:
                # Handle errors
                data = response.json()
                if 'Error' in data and 'message' in data["Error"] and data["Error"]["message"] is not None:
                    print("Error: " + data["Error"]["message"])
                else:
                    print("Error: Preview Order API service error")
        else:
            # Handle errors
            data = response.json()
            if 'Error' in data and 'message' in data["Error"] and data["Error"]["message"] is not None:
                print("Error: " + data["Error"]["message"])
                
        return  

    def place_market_order(self,order,preview_id):
        
        #Function to place market order
        
        payload = """<PlaceOrderRequest>
                       <orderType>EQ</orderType>
                       <clientOrderId>{0}</clientOrderId>
                        <PreviewIds>
                            <previewId>{1}</previewId>
                        </PreviewIds>
                       <Order>
                           <allOrNone>false</allOrNone>
                           <priceType>{2}</priceType>
                           <orderTerm>{3}</orderTerm>
                           <marketSession>REGULAR</marketSession>
                           <stopPrice></stopPrice>
                           <limitPrice>{4}</limitPrice>
                           <Instrument>
                               <Product>
                                   <securityType>EQ</securityType>
                                   <symbol>{5}</symbol>
                               </Product>
                               <orderAction>{6}</orderAction>
                               <quantityType>QUANTITY</quantityType>
                               <quantity>{7}</quantity>
                           </Instrument>
                       </Order>
                   </PlaceOrderRequest>"""
        payload = payload.format(order["client_order_id"], preview_id,
                                 order["price_type"], order["order_term"],
                                 order["limit_price"], order["symbol"], order["order_action"], order["quantity"])

        # URL for the API endpoint
        url = self.base_url + "/v1/accounts/" + account_ID_key + "/orders/place.json"

        # Add parameters and header information
        headers = {"Content-Type": "application/xml", "consumerKey": config["DEFAULT"]["CONSUMER_KEY"]}

        # Make API call for POST request
        response = self.session.post(url, header_auth=True, headers=headers, data=payload)
        logger.debug("Request Header: %s", response.request.headers)
        logger.debug("Request payload: %s", payload)

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()

            messages = data['PlaceOrderResponse']['Order'][0]['messages']['Message']
            for message in messages:
                description = message['description']
                print(description)

        else:
            # Handle errors
            data = response.json()
            if 'Error' in data and 'message' in data["Error"] and data["Error"]["message"] is not None:
                print("Error: " + data["Error"]["message"])
                
        return    

    def preview_and_place_limit_order(self,term,symbol,order_action,limit_price,quantity):
        
        #Function to structure limit order data
        #Then excecutes function that places limit order (next function)

        order = {"price_type": "LIMIT",
                 "order_term": term,
                 "symbol": symbol,
                 "order_action": order_action,
                 "limit_price":limit_price,
                 "quantity": quantity}
        order["client_order_id"] = random.randint(1000000000, 9999999999) #Unique identifier for order

#        price_type_options = ["MARKET", "LIMIT"]
#        order_term_options = ["GOOD_FOR_DAY", "IMMEDIATE_OR_CANCEL", "FILL_OR_KILL"]
#        order_action_options = ["BUY", "SELL", "BUY_TO_COVER", "SELL_SHORT"]

        # URL for the API endpoint
        url = self.base_url + "/v1/accounts/" + account_ID_key + "/orders/preview.json"
        headers = {"Content-Type": "application/xml", "consumerKey": config["DEFAULT"]["CONSUMER_KEY"]}

        # Add payload for POST Request
        payload = """<PreviewOrderRequest>
                       <orderType>EQ</orderType>
                       <clientOrderId>{0}</clientOrderId>
                       <Order>
                           <allOrNone>false</allOrNone>
                           <priceType>{1}</priceType>
                           <orderTerm>{2}</orderTerm>
                           <marketSession>REGULAR</marketSession>
                           <stopPrice></stopPrice>
                           <limitPrice>{3}</limitPrice>
                           <Instrument>
                               <Product>
                                   <securityType>EQ</securityType>
                                   <symbol>{4}</symbol>
                               </Product>
                               <orderAction>{5}</orderAction>
                               <quantityType>QUANTITY</quantityType>
                               <quantity>{6}</quantity>
                           </Instrument>
                       </Order>
                   </PreviewOrderRequest>"""

        payload = payload.format(order["client_order_id"], 
                                 order["price_type"], order["order_term"],
                                 order["limit_price"], order["symbol"], order["order_action"], order["quantity"])

        # Make API call for POST request
        response = self.session.post(url, header_auth=True, headers=headers, data=payload)
        logger.debug("Request Header: %s", response.request.headers)
        logger.debug("Request payload: %s", payload)

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()
            print("\nPreview Order:")

            if data is not None and "PreviewOrderResponse" in data and "PreviewIds" in data["PreviewOrderResponse"]:
                for previewids in data["PreviewOrderResponse"]["PreviewIds"]:
                    print("Preview ID: " + str(previewids["previewId"]))
                    preview_id = previewids['previewId']
                    self.place_limit_order(order,preview_id)  #RUNNING ORDER EXECUTION FUNCTION HERE
            else:
                # Handle errors
                data = response.json()
                if 'Error' in data and 'message' in data["Error"] and data["Error"]["message"] is not None:
                    print("Error: " + data["Error"]["message"])
                else:
                    print("Error: Preview Order API service error")

            if data is not None and "PreviewOrderResponse" in data and "Order" in data["PreviewOrderResponse"]:
                for orders in data["PreviewOrderResponse"]["Order"]:
                    order["limitPrice"] = orders["limitPrice"]

                    if orders is not None and "Instrument" in orders:
                        for instrument in orders["Instrument"]:
                            if instrument is not None and "orderAction" in instrument:
                                print("Action: " + instrument["orderAction"])
                            if instrument is not None and "quantity" in instrument:
                                print("Quantity: " + str(instrument["quantity"]))
                            if instrument is not None and "Product" in instrument \
                                    and "symbol" in instrument["Product"]:
                                print("Symbol: " + instrument["Product"]["symbol"])
                            if instrument is not None and "symbolDescription" in instrument:
                                print("Description: " + str(instrument["symbolDescription"]))

                if orders is not None and "priceType" in orders and "limitPrice" in orders:
                    print("Price Type: " + orders["priceType"])
                    if orders["priceType"] == "MARKET":
                        print("Price: MKT")
                    else:
                        print("Price: " + str(orders["limitPrice"]))
                if orders is not None and "orderTerm" in orders:
                    print("Duration: " + orders["orderTerm"])
                if orders is not None and "estimatedCommission" in orders:
                    print("Estimated Commission: " + str(orders["estimatedCommission"]))
                if orders is not None and "estimatedTotalAmount" in orders:
                    print("Estimated Total Cost: " + str(orders["estimatedTotalAmount"]))
            else:
                # Handle errors
                data = response.json()
                if 'Error' in data and 'message' in data["Error"] and data["Error"]["message"] is not None:
                    print("Error: " + data["Error"]["message"])
                else:
                    print("Error: Preview Order API service error")
        else:
            # Handle errors
            data = response.json()
            if 'Error' in data and 'message' in data["Error"] and data["Error"]["message"] is not None:
                print("Error: " + data["Error"]["message"])
        return    

    def place_limit_order(self,order,preview_id):
        
        #Function to execute limit order

        payload = """<PlaceOrderRequest>
                       <orderType>EQ</orderType>
                       <clientOrderId>{0}</clientOrderId>
                        <PreviewIds>
                            <previewId>{1}</previewId>
                        </PreviewIds>
                       <Order>
                           <allOrNone>false</allOrNone>
                           <priceType>{2}</priceType>
                           <orderTerm>{3}</orderTerm>
                           <marketSession>REGULAR</marketSession>
                           <stopPrice></stopPrice>
                           <limitPrice>{4}</limitPrice>
                           <Instrument>
                               <Product>
                                   <securityType>EQ</securityType>
                                   <symbol>{5}</symbol>
                               </Product>
                               <orderAction>{6}</orderAction>
                               <quantityType>QUANTITY</quantityType>
                               <quantity>{7}</quantity>
                           </Instrument>
                       </Order>
                   </PlaceOrderRequest>"""
        payload = payload.format(order["client_order_id"], preview_id,
                                 order["price_type"], order["order_term"],
                                 order["limit_price"], order["symbol"], order["order_action"], order["quantity"])

        # URL for the API endpoint
        url = self.base_url + "/v1/accounts/" + account_ID_key + "/orders/place.json"

        # Add parameters and header information
        headers = {"Content-Type": "application/xml", "consumerKey": config["DEFAULT"]["CONSUMER_KEY"]}

        # Make API call for POST request
        response = self.session.post(url, header_auth=True, headers=headers, data=payload)
        logger.debug("Request Header: %s", response.request.headers)
        logger.debug("Request payload: %s", payload)

        # Handle and parse response
        if response is not None and response.status_code == 200:
            parsed = json.loads(response.text)
            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
            data = response.json()

            messages = data['PlaceOrderResponse']['Order'][0]['messages']['Message']
            for message in messages:
                description = message['description']
                print(description)

        else:
            # Handle errors
            data = response.json()
            if 'Error' in data and 'message' in data["Error"] and data["Error"]["message"] is not None:
                print("Error: " + data["Error"]["message"])
                
        return

    def cancel_orders(self,account_ID_key):
        
        #Calls cancel order API to cancel an existing order
        
        while True:
            # Display a list of Open Orders
            # URL for the API endpoint
            url = self.base_url + "/v1/accounts/" + account_ID_key + "/orders.json"

            # Add parameters and header information
            params_open = {"status": "OPEN"}
            headers = {"consumerkey": config["DEFAULT"]["CONSUMER_KEY"]}

            # Make API call for GET request
            response_open = self.session.get(url, header_auth=True, params=params_open, headers=headers)

            logger.debug("Request Header: %s", response_open.request.headers)
            logger.debug("Response Body: %s", response_open.text)

            print("\nOpen Orders: ")
            # Handle and parse response
            if response_open.status_code == 204:
                logger.debug(response_open)
                print("None")
                return
                
            elif response_open.status_code == 200:
                parsed = json.loads(response_open.text)
                logger.debug(json.dumps(parsed, indent=4, sort_keys=True))
                data = response_open.json()

                order_list = []
                count = 1
                if data is not None and "OrdersResponse" in data and "Order" in data["OrdersResponse"]:
                    for order in data["OrdersResponse"]["Order"]:
                        if order is not None and "OrderDetail" in order:
                            for details in order["OrderDetail"]:
                                if details is not None and "Instrument" in details:
                                    for instrument in details["Instrument"]:
                                        order_str = ""
                                        order_obj = {"price_type": None,
                                                     "order_term": None,
                                                     "order_indicator": None,
                                                     "order_type": None,
                                                     "security_type": None,
                                                     "symbol": None,
                                                     "order_action": None,
                                                     "quantity": None}
                                        if order is not None and 'orderType' in order:
                                            order_obj["order_type"] = order["orderType"]

                                        if order is not None and 'orderId' in order:
                                            order_str += "Order #" + str(order["orderId"]) + " : "
                                        if instrument is not None and 'Product' in instrument and 'securityType' \
                                                in instrument["Product"]:
                                            order_str += "Type: " + instrument["Product"]["securityType"] + " | "
                                            order_obj["security_type"] = instrument["Product"]["securityType"]

                                        if instrument is not None and 'orderAction' in instrument:
                                            order_str += "Order Type: " + instrument["orderAction"] + " | "
                                            order_obj["order_action"] = instrument["orderAction"]

                                        if instrument is not None and 'orderedQuantity' in instrument:
                                            order_str += "Quantity(Exec/Entered): " + str(
                                                "{:,}".format(instrument["orderedQuantity"])) + " | "
                                            order_obj["quantity"] = instrument["orderedQuantity"]

                                        if instrument is not None and 'Product' in instrument and 'symbol' \
                                                in instrument["Product"]:
                                            order_str += "Symbol: " + instrument["Product"]["symbol"] + " | "
                                            order_obj["symbol"] = instrument["Product"]["symbol"]

                                        if details is not None and 'priceType' in details:
                                            order_str += "Price Type: " + details["priceType"] + " | "
                                            order_obj["price_type"] = details["priceType"]

                                        if details is not None and 'orderTerm' in details:
                                            order_str += "Term: " + details["orderTerm"] + " | "
                                            order_obj["order_term"] = details["orderTerm"]

                                        if details is not None and 'limitPrice' in details:
                                            order_str += "Price: " + str(
                                                '${:,.2f}'.format(details["limitPrice"])) + " | "
                                            order_obj["limitPrice"] = details["limitPrice"]

                                        if instrument is not None and 'filledQuantity' in instrument:
                                            order_str += "Quantity Executed: " \
                                                         + str("{:,}".format(instrument["filledQuantity"])) + " | "
                                            order_obj["quantity"] = instrument["filledQuantity"]

                                        if instrument is not None and "averageExecutionPrice" in instrument:
                                            order_str += "Price Executed: " + str(
                                                '${:,.2f}'.format(instrument["averageExecutionPrice"])) + " | "

                                        if details is not None and 'status' in details:
                                            order_str += "Status: " + details["status"]

                                        print(str(count) + ")\t" + order_str)
                                        count = 1 + count
                                        order_list.append(order["orderId"])


                    # URL for the API endpoint
                    url = self.base_url + "/v1/accounts/" + account_ID_key + "/orders/cancel.json"

                    # Add parameters and header information
                    headers = {"Content-Type": "application/xml", "consumerKey": config["DEFAULT"]["CONSUMER_KEY"]}

                    # Add payload for POST Request
                    payload = """<CancelOrderRequest>
                                    <orderId>{0}</orderId>
                                </CancelOrderRequest>
                               """
                    for order in order_list:  
                        payload = payload.format(order)

                        # Add payload for PUT Request
                        response = self.session.put(url, header_auth=True, headers=headers, data=payload)
                        logger.debug("Request Header: %s", response.request.headers)
                        logger.debug("Request payload: %s", payload)

                        time.sleep(5) #wait 5 seconds (avoids repeat cancellations)
                        
                        # Handle and parse response
                        if response is not None and response.status_code == 200:
                            parsed = json.loads(response.text)
                            logger.debug("Response Body: %s", json.dumps(parsed, indent=4, sort_keys=True))
                            data = response.json()
                            if data is not None and "CancelOrderResponse" in data \
                                    and "orderId" in data["CancelOrderResponse"]:
                                print("\nOrder number #" + str(
                                    data["CancelOrderResponse"]["orderId"]) + " successfully Cancelled.")
                            else:
                                # Handle errors
                                logger.debug("Response Headers: %s", response.headers)
                                logger.debug("Response Body: %s", response.text)
                                data = response.json()
                                if 'Error' in data and 'message' in data["Error"] \
                                        and data["Error"]["message"] is not None:
                                    print("Error: " + data["Error"]["message"])
                                else:
                                    print("Error: Cancel Order API service error")
                        else:
                            # Handle errors
                            logger.debug("Response Headers: %s", response.headers)
                            logger.debug("Response Body: %s", response.text)
                            data = response.json()
                            if 'Error' in data and 'message' in data["Error"] and data["Error"]["message"] is not None:
                                print("Error: " + data["Error"]["message"])
                            else:
                                print("Error: Cancel Order API service error")
                        break


            else:
                # Handle errors
                logger.debug("Response Body: %s", response_open.text)
                if response_open is not None and response_open.headers['Content-Type'] == 'application/json' \
                        and "Error" in response_open.json() and "message" in response_open.json()["Error"] \
                        and response_open.json()["Error"]["message"] is not None:
                    print("Error: " + response_open.json()["Error"]["message"])
                else:
                    print("Error: Balance API service error")
                break
        else:
            # Handle errors
            logger.debug("Response Body: %s", response_open.text)
            if response_open is not None and response_open.headers['Content-Type'] == 'application/json' \
                    and "Error" in response_open.json() and "message" in response_open.json()["Error"] \
                    and response_open.json()["Error"]["message"] is not None:
                print("Error: " + response_open.json()["Error"]["message"])
            else:
                print("Error: Balance API service error")
        return



In [19]:
#Start with canceling and existing orders (clean slate to start excecuting system trades)
orders = Order(session, account_ID_key, base_url)
orders.cancel_orders(account_ID_key) #cancels all outstanding orders




Open Orders: 
None


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 74, in emit
    self.doRollover()
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 179, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 115, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log' -> 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log.1'
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\reedx\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\reedx\anaconda3\Lib\site-packages\traitlets\config\application.

In [16]:
#Running sell orders
current_portfolio_df = accounts.portfolio(account_ID_key) #find current portfolio

#Ensure system has up to date current portfolio positions
current_portfolio_df = accounts.portfolio(account_ID_key) #recalc current portfolio

try:
    #First check if the system signaled for any transactions
    if transaction_df.empty:
        print("No transactions today")

    #To best excecute try selling at ask, then middle of spread, then bid, then bid, then bid, then market
    else: 

        orders = Order(session, account_ID_key, base_url) #establish orders class
        orders.cancel_orders(account_ID_key) #cancels all outstanding orders
        time.sleep(5) #wait 5 seconds for cancel orders to run
        sell_transaction_df = transaction_df[transaction_df['order'].str.contains('Sell',case=False)]

        if sell_transaction_df.empty:
            print('No sell orders today')

        else: #try selling at ask
            for index,row in sell_transaction_df.iterrows():
                ticker = row['Ticker']
                limit_price = row['ask']
                quantity = row['shares_to_trade_rounded']
                orders.preview_and_place_limit_order("GOOD_FOR_DAY",ticker,'SELL',limit_price,quantity)

            #wait 30 seconds, cancel orders, check portfolio and calc trades
            time.sleep(30) #give 30 seconds for orders to execute
            orders.cancel_orders(account_ID_key) #cancels all outstanding orders
            time.sleep(5) #wait 5 seconds for cancel orders to run
            
            #check if trade orders executed by comparing portfolio to previous "current_portfolio_df"
            portfolio_after_trade_df = accounts.portfolio(account_ID_key) #recalc current portfolio
            before_after_portfolio_df = pd.merge(current_portfolio_df, portfolio_after_trade_df, on = 'Ticker', how = 'outer', suffixes = ('_prior','_after'))
            before_after_portfolio_df['Quantity_prior'] = before_after_portfolio_df['Quantity_prior'].fillna(0) #will have na if no position before trade, enter as 0
            before_after_portfolio_df['Quantity_after'] = before_after_portfolio_df['Quantity_after'].fillna(0) #will have na if no position after trade, enter as 0                                     
            before_after_portfolio_df['Trades'] = before_after_portfolio_df['Quantity_prior'] -  before_after_portfolio_df['Quantity_after']
            
            merged_df = pd.merge(transaction_df, before_after_portfolio_df, on='Ticker', how = 'left') #merge with transaction df, keeping transaction_df rows only
            print(merged_df)
            merged_df['Trades'] = merged_df['Trades'].fillna(0) #if value in transaction_df was not in before_after_portfolio_df, then value in 'Trades' = nan, need to replace that with 0 b/c it was not traded
            merged_df = merged_df[~merged_df['Trades'].ne(0)] #remove rows where trades <>0 that identifies that a trade went through #double negative ~ (not) & ne (not equal)
            transaction_df = merged_df
            transaction_df = transaction_df[['Ticker','Quantity','weight','last_trade','bid','ask','order','shares_to_trade_rounded','transaction_value_rounded']]          
            sell_transaction_df = transaction_df[transaction_df['order'].str.contains('Sell',case=False)]
            
            if sell_transaction_df.empty:
                print('No more sell orders')

            else: #try executing mid spread
                for index,row in sell_transaction_df.iterrows():
                    ticker = row['Ticker']
                    limit_price = round((row['ask'] + row['bid'])/2,2)
                    quantity = row['shares_to_trade_rounded']
                    orders.preview_and_place_limit_order("GOOD_FOR_DAY",ticker,'SELL',limit_price,quantity)

                #wait 30 seconds, cancel orders, check portfolio and calc trades
                time.sleep(30) #give 30 seconds for orders to execute
                orders.cancel_orders(account_ID_key) #cancels all outstanding orders
                time.sleep(5) #wait 5 seconds for cancel orders to run

                #check if trade orders executed by comparing portfolio to previous "current_portfolio_df"
                portfolio_after_trade_df = accounts.portfolio(account_ID_key) #recalc current portfolio
                before_after_portfolio_df = pd.merge(current_portfolio_df, portfolio_after_trade_df, on = 'Ticker', how = 'outer', suffixes = ('_prior','_after'))
                before_after_portfolio_df['Quantity_prior'] = before_after_portfolio_df['Quantity_prior'].fillna(0) #will have na if no position before trade, enter as 0
                before_after_portfolio_df['Quantity_after'] = before_after_portfolio_df['Quantity_after'].fillna(0) #will have na if no position after trade, enter as 0                                     
                before_after_portfolio_df['Trades'] = before_after_portfolio_df['Quantity_prior'] -  before_after_portfolio_df['Quantity_after']

                merged_df = pd.merge(transaction_df, before_after_portfolio_df, on='Ticker', how = 'left') #merge with transaction df, keeping transaction_df rows only
                merged_df['Trades'] = merged_df['Trades'].fillna(0) #if value in transaction_df was not in before_after_portfolio_df, then value in 'Trades' = nan, need to replace that with 0 b/c it was not traded
                merged_df = merged_df[~merged_df['Trades'].ne(0)] #remove rows where trades <>0 that identifies that a trade went through #double negative ~ (not) & ne (not equal)
                transaction_df = merged_df
                transaction_df = transaction_df[['Ticker','Quantity','weight','last_trade','bid','ask','order','shares_to_trade_rounded','transaction_value_rounded']]
                sell_transaction_df = transaction_df[transaction_df['order'].str.contains('Sell',case=False)]

                if sell_transaction_df.empty:
                    print('No more sell orders')

                else: #try executing at bid (1st time)
                    for index,row in sell_transaction_df.iterrows():
                        ticker = row['Ticker']
                        limit_price = row['bid']
                        quantity = row['shares_to_trade_rounded']
                        orders.preview_and_place_limit_order("GOOD_FOR_DAY",ticker,'SELL',limit_price,quantity)

                    #wait 30 seconds, cancel orders, check portfolio and calc trades
                    time.sleep(30) #give 30 seconds for orders to execute
                    orders.cancel_orders(account_ID_key) #cancels all outstanding orders
                    time.sleep(5) #wait 5 seconds for cancel orders to run

                    #check if trade orders executed by comparing portfolio to previous "current_portfolio_df"
                    portfolio_after_trade_df = accounts.portfolio(account_ID_key) #recalc current portfolio
                    before_after_portfolio_df = pd.merge(current_portfolio_df, portfolio_after_trade_df, on = 'Ticker', how = 'outer', suffixes = ('_prior','_after'))
                    before_after_portfolio_df['Quantity_prior'] = before_after_portfolio_df['Quantity_prior'].fillna(0) #will have na if no position before trade, enter as 0
                    before_after_portfolio_df['Quantity_after'] = before_after_portfolio_df['Quantity_after'].fillna(0) #will have na if no position after trade, enter as 0                                     
                    before_after_portfolio_df['Trades'] = before_after_portfolio_df['Quantity_prior'] -  before_after_portfolio_df['Quantity_after']

                    merged_df = pd.merge(transaction_df, before_after_portfolio_df, on='Ticker', how = 'left') #merge with transaction df, keeping transaction_df rows only
                    merged_df['Trades'] = merged_df['Trades'].fillna(0) #if value in transaction_df was not in before_after_portfolio_df, then value in 'Trades' = nan, need to replace that with 0 b/c it was not traded
                    merged_df = merged_df[~merged_df['Trades'].ne(0)] #remove rows where trades <>0 that identifies that a trade went through #double negative ~ (not) & ne (not equal)
                    transaction_df = merged_df
                    transaction_df = transaction_df[['Ticker','Quantity','weight','last_trade','bid','ask','order','shares_to_trade_rounded','transaction_value_rounded']]
                    sell_transaction_df = transaction_df[transaction_df['order'].str.contains('Sell',case=False)]

                    if sell_transaction_df.empty:
                        print('No more sell orders')

                    else: #try executing at bid (2nd time)
                        for index,row in sell_transaction_df.iterrows():
                            ticker = row['Ticker']
                            limit_price = row['bid']
                            quantity = row['shares_to_trade_rounded']
                            orders.preview_and_place_limit_order("GOOD_FOR_DAY",ticker,'SELL',limit_price,quantity)

                        #wait 30 seconds, cancel orders, check portfolio and calc trades
                        time.sleep(30) #give 30 seconds for orders to execute
                        orders.cancel_orders(account_ID_key) #cancels all outstanding orders
                        time.sleep(5) #wait 5 seconds for cancel orders to run
 
                        #check if trade orders executed by comparing portfolio to previous "current_portfolio_df"
                        portfolio_after_trade_df = accounts.portfolio(account_ID_key) #recalc current portfolio
                        before_after_portfolio_df = pd.merge(current_portfolio_df, portfolio_after_trade_df, on = 'Ticker', how = 'outer', suffixes = ('_prior','_after'))
                        before_after_portfolio_df['Quantity_prior'] = before_after_portfolio_df['Quantity_prior'].fillna(0) #will have na if no position before trade, enter as 0
                        before_after_portfolio_df['Quantity_after'] = before_after_portfolio_df['Quantity_after'].fillna(0) #will have na if no position after trade, enter as 0                                     
                        before_after_portfolio_df['Trades'] = before_after_portfolio_df['Quantity_prior'] -  before_after_portfolio_df['Quantity_after']

                        merged_df = pd.merge(transaction_df, before_after_portfolio_df, on='Ticker', how = 'left') #merge with transaction df, keeping transaction_df rows only
                        merged_df['Trades'] = merged_df['Trades'].fillna(0) #if value in transaction_df was not in before_after_portfolio_df, then value in 'Trades' = nan, need to replace that with 0 b/c it was not traded
                        merged_df = merged_df[~merged_df['Trades'].ne(0)] #remove rows where trades <>0 that identifies that a trade went through #double negative ~ (not) & ne (not equal)
                        transaction_df = merged_df
                        transaction_df = transaction_df[['Ticker','Quantity','weight','last_trade','bid','ask','order','shares_to_trade_rounded','transaction_value_rounded']]
                        sell_transaction_df = transaction_df[transaction_df['order'].str.contains('Sell',case=False)]

                        if sell_transaction_df.empty:
                            print('No more sell orders')

                        else: #try executing at bid (3rd time)
                            for index,row in sell_transaction_df.iterrows():
                                ticker = row['Ticker']
                                limit_price = row['bid']
                                quantity = row['shares_to_trade_rounded']
                                orders.preview_and_place_limit_order("GOOD_FOR_DAY",ticker,'SELL',limit_price,quantity)


                            #wait 30 seconds, cancel orders, check portfolio and calc trades
                            time.sleep(30) #give 30 seconds for orders to execute
                            orders.cancel_orders(account_ID_key) #cancels all outstanding orders
                            time.sleep(5) #wait 5 seconds for cancel orders to run

                            #check if trade orders executed by comparing portfolio to previous "current_portfolio_df"
                            portfolio_after_trade_df = accounts.portfolio(account_ID_key) #recalc current portfolio
                            before_after_portfolio_df = pd.merge(current_portfolio_df, portfolio_after_trade_df, on = 'Ticker', how = 'outer', suffixes = ('_prior','_after'))
                            before_after_portfolio_df['Quantity_prior'] = before_after_portfolio_df['Quantity_prior'].fillna(0) #will have na if no position before trade, enter as 0
                            before_after_portfolio_df['Quantity_after'] = before_after_portfolio_df['Quantity_after'].fillna(0) #will have na if no position after trade, enter as 0                                     
                            before_after_portfolio_df['Trades'] = before_after_portfolio_df['Quantity_prior'] -  before_after_portfolio_df['Quantity_after']

                            merged_df = pd.merge(transaction_df, before_after_portfolio_df, on='Ticker', how = 'left') #merge with transaction df, keeping transaction_df rows only
                            merged_df['Trades'] = merged_df['Trades'].fillna(0) #if value in transaction_df was not in before_after_portfolio_df, then value in 'Trades' = nan, need to replace that with 0 b/c it was not traded
                            merged_df = merged_df[~merged_df['Trades'].ne(0)] #remove rows where trades <>0 that identifies that a trade went through #double negative ~ (not) & ne (not equal)
                            transaction_df = merged_df
                            transaction_df = transaction_df[['Ticker','Quantity','weight','last_trade','bid','ask','order','shares_to_trade_rounded','transaction_value_rounded']]
                            sell_transaction_df = transaction_df[transaction_df['order'].str.contains('Sell',case=False)]
                            

                            if sell_transaction_df.empty:
                                print('No more sell orders')

                            else: #try executing at market
                                for index,row in sell_transaction_df.iterrows():
                                    ticker = row['Ticker']
                                    quantity = row['shares_to_trade_rounded']
                                    orders.preview_and_place_market_order(ticker,'SELL',quantity)

                                time.sleep(300) #give 300 seconds for orders to execute
                                orders.cancel_orders(account_ID_key) #cancels all outstanding orders
                                time.sleep(5) #wait 5 seconds for cancel orders to run
                                
                                #check if trade orders executed by comparing portfolio to previous "current_portfolio_df"
                                portfolio_after_trade_df = accounts.portfolio(account_ID_key) #recalc current portfolio
                                before_after_portfolio_df = pd.merge(current_portfolio_df, portfolio_after_trade_df, on = 'Ticker', how = 'outer', suffixes = ('_prior','_after'))
                                before_after_portfolio_df['Quantity_prior'] = before_after_portfolio_df['Quantity_prior'].fillna(0) #will have na if no position before trade, enter as 0
                                before_after_portfolio_df['Quantity_after'] = before_after_portfolio_df['Quantity_after'].fillna(0) #will have na if no position after trade, enter as 0                                     
                                before_after_portfolio_df['Trades'] = before_after_portfolio_df['Quantity_prior'] -  before_after_portfolio_df['Quantity_after']

                                merged_df = pd.merge(transaction_df, before_after_portfolio_df, on='Ticker', how = 'left') #merge with transaction df, keeping transaction_df rows only
                                merged_df['Trades'] = merged_df['Trades'].fillna(0) #if value in transaction_df was not in before_after_portfolio_df, then value in 'Trades' = nan, need to replace that with 0 b/c it was not traded
                                merged_df = merged_df[~merged_df['Trades'].ne(0)] #remove rows where trades <>0 that identifies that a trade went through #double negative ~ (not) & ne (not equal)
                                transaction_df = merged_df
                                transaction_df = transaction_df[['Ticker','Quantity','weight','last_trade','bid','ask','order','shares_to_trade_rounded','transaction_value_rounded']]
                                sell_transaction_df = transaction_df[transaction_df['order'].str.contains('Sell',case=False)]

                                if sell_transaction_df.empty:
                                    orders.cancel_orders(account_ID_key) #cancels all outstanding orders                        
                                    print('No more sell orders')

                                else:
                                    orders.cancel_orders(account_ID_key) #cancels all outstanding orders
                                    print("ORDERS UNABLE TO EXECUTE:")
                                    print(sell_transaction_df)

except:
    error = pd.DataFrame({'date':[end_date.strftime('%Y-%m-%d')],
                        'error': ['Error in sell order execution']})
    error.set_index('date',inplace=True)
    error_combined = pd.concat([error_df,error], ignore_index = False)
    
    writer = pd.ExcelWriter(error_file)
    error_combined.to_excel(writer, index=True)
    writer.close()
    raise ValueError('Error')    


C:\Users\reedx\AppData\Local\Temp\ipykernel_7068\3192054314.py:327: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_portfolio_df = pd.concat([current_portfolio_df,new_row], ignore_index = True)
C:\Users\reedx\AppData\Local\Temp\ipykernel_7068\3192054314.py:327: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_portfolio_df = pd.concat([current_portfolio_df,new_row], ignore_index = True)



Open Orders: 
None

Preview Order:
Preview ID: 2077329378106
200|The market was closed when we received your order. It has been entered into our system and will be reviewed prior to market open on the next regular trading day. After market open, please check to make sure your order was accepted.
Action: SELL
Quantity: 18
Symbol: DJP
Description: IPATH BLOOMBERG COMMODITY INDEX TOTAL RETURN ETN
Price Type: LIMIT
Price: 34.46
Duration: GOOD_FOR_DAY
Estimated Commission: 0
Estimated Total Cost: -620.2597

Preview Order:
Preview ID: 2077333984106
400|Average Cost methodology to determine your cost basis is an available option. To update your cost basis method, please click here.
200|The market was closed when we received your order. It has been entered into our system and will be reviewed prior to market open on the next regular trading day. After market open, please check to make sure your order was accepted.
Action: SELL
Quantity: 7
Symbol: XLU
Description: UTILITIES SELECT SECTOR SPDR 

C:\Users\reedx\AppData\Local\Temp\ipykernel_7068\3192054314.py:327: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_portfolio_df = pd.concat([current_portfolio_df,new_row], ignore_index = True)


   Ticker Quantity  current_weight  weight  ending weight estimate  \
0    IBIT      NaN        0.000000     0.1            9.742943e-02   
1    BITQ      NaN        0.000000     0.1            9.990885e-02   
2    ARKK      NaN        0.000000     0.1            9.812087e-02   
3    ARKX      NaN        0.000000     0.1            1.004373e-01   
4    JETS      NaN        0.000000     0.1            9.916493e-02   
5    IBLC      NaN        0.000000     0.1            1.001786e-01   
6     EIS      NaN        0.000000     0.1            9.740365e-02   
7     XTL      NaN        0.000000     0.1            1.031819e-01   
8     XLY      NaN        0.000000     0.1            8.522658e-02   
9    CLOU      NaN        0.000000     0.1            1.006684e-01   
10    DJP       18        0.056197     0.0           -9.206916e-09   
11    XLU        7        0.050714     0.0            1.046705e-17   
12    XLP       14        0.099895     0.0            0.000000e+00   
13    XLE        5  

C:\Users\reedx\AppData\Local\Temp\ipykernel_7068\3192054314.py:327: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_portfolio_df = pd.concat([current_portfolio_df,new_row], ignore_index = True)


...

C:\Users\reedx\AppData\Local\Temp\ipykernel_7068\3192054314.py:327: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_portfolio_df = pd.concat([current_portfolio_df,new_row], ignore_index = True)



Preview Order:
Preview ID: 2077335183106
200|The market was closed when we received your order. It has been entered into our system and will be reviewed prior to market open on the next regular trading day. After market open, please check to make sure your order was accepted.
Action: SELL
Quantity: 18
Symbol: DJP
Description: IPATH BLOOMBERG COMMODITY INDEX TOTAL RETURN ETN
Price Type: LIMIT
Price: 33.77
Duration: GOOD_FOR_DAY
Estimated Commission: 0
Estimated Total Cost: -607.8401

Preview Order:
Preview ID: 2077346789106
400|Average Cost methodology to determine your cost basis is an available option. To update your cost basis method, please click here.
200|The market was closed when we received your order. It has been entered into our system and will be reviewed prior to market open on the next regular trading day. After market open, please check to make sure your order was accepted.
Action: SELL
Quantity: 7
Symbol: XLU
Description: UTILITIES SELECT SECTOR SPDR FUND
Price Type: LIM

C:\Users\reedx\AppData\Local\Temp\ipykernel_7068\3192054314.py:327: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_portfolio_df = pd.concat([current_portfolio_df,new_row], ignore_index = True)



Preview Order:
Preview ID: 2077333860106
200|The market was closed when we received your order. It has been entered into our system and will be reviewed prior to market open on the next regular trading day. After market open, please check to make sure your order was accepted.
Action: SELL
Quantity: 18
Symbol: DJP
Description: IPATH BLOOMBERG COMMODITY INDEX TOTAL RETURN ETN
Price Type: MARKET
Price: MKT
Duration: GOOD_FOR_DAY
Estimated Commission: 0
Estimated Total Cost: -610.36

Preview Order:
Preview ID: 2077334383106
400|Average Cost methodology to determine your cost basis is an available option. To update your cost basis method, please click here.
200|The market was closed when we received your order. It has been entered into our system and will be reviewed prior to market open on the next regular trading day. After market open, please check to make sure your order was accepted.
Action: SELL
Quantity: 7
Symbol: XLU
Description: UTILITIES SELECT SECTOR SPDR FUND
Price Type: MARKET

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 74, in emit
    self.doRollover()
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 179, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 115, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log' -> 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log.1'
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\reedx\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\reedx\anaconda3\Lib\site-packages\traitlets\config\application.

In [17]:
#Running buy orders
current_portfolio_df = accounts.portfolio(account_ID_key) #recalc current portfolio

#Ensure system has up to date current portfolio positions
current_portfolio_df = accounts.portfolio(account_ID_key) #recalc current portfolio


try:
    #First check if the system signaled for any transactions
    if transaction_df.empty:
        print("No transactions today")

    else: #For best excecution, try buying at bid, then middle of spread, then ask, then ask, then ask, then market

        orders = Order(session, account_ID_key, base_url) #establish orders class

        orders.cancel_orders(account_ID_key) #cancels all outstanding orders
        time.sleep(5) #wait 5 seconds for cancel orders to run
        buy_transaction_df = transaction_df[transaction_df['order'].str.contains('Buy',case=False)]
        print('buy transaction df 1')
        print(buy_transaction_df)

        if buy_transaction_df.empty:
            print('No buy orders today')

        else:

            for index,row in buy_transaction_df.iterrows(): #try buying at bid
                ticker = row['Ticker']
                limit_price = row['bid']
                quantity = row['shares_to_trade_rounded']
                orders.preview_and_place_limit_order("GOOD_FOR_DAY",ticker,'BUY',limit_price,quantity)

            #wait 30 seconds, cancel orders, check portfolio and calc trades
            time.sleep(30) #give 30 seconds for orders to execute
            orders.cancel_orders(account_ID_key) #cancels all outstanding orders
            time.sleep(5) #wait 5 seconds for cancel orders to run

            #check if trade orders executed by comparing portfolio to previous "current_portfolio_df"
            portfolio_after_trade_df = accounts.portfolio(account_ID_key) #recalc current portfolio
            print('portfolio after trade')
            print(portfolio_after_trade_df)
            
            before_after_portfolio_df = pd.merge(current_portfolio_df, portfolio_after_trade_df, on = 'Ticker', how = 'outer', suffixes = ('_prior','_after'))
            before_after_portfolio_df['Quantity_prior'] = before_after_portfolio_df['Quantity_prior'].fillna(0) #will have na if no position before trade, enter as 0
            before_after_portfolio_df['Quantity_after'] = before_after_portfolio_df['Quantity_after'].fillna(0) #will have na if no position after trade, enter as 0                                     
            before_after_portfolio_df['Trades'] = before_after_portfolio_df['Quantity_prior'] -  before_after_portfolio_df['Quantity_after']
            print('before after portfolio 1')
            print(before_after_portfolio_df)
            
            merged_df = pd.merge(transaction_df, before_after_portfolio_df, on='Ticker', how = 'left') #merge with transaction df, keeping transaction_df rows only
            merged_df['Trades'] = merged_df['Trades'].fillna(0) #if value in transaction_df was not in before_after_portfolio_df, then value in 'Trades' = nan, need to replace that with 0 b/c it was not traded
            merged_df = merged_df[~merged_df['Trades'].ne(0)] #remove rows where trades <>0 that identifies that a trade went through #double negative ~ (not) & ne (not equal)
            print('merged df')
            print(merged_df)
            
            transaction_df = merged_df
            transaction_df = transaction_df[['Ticker','Quantity','weight','last_trade','bid','ask','order','shares_to_trade_rounded','transaction_value_rounded']]
            buy_transaction_df = transaction_df[transaction_df['order'].str.contains('Buy',case=False)]
            print('buy transaction df 1.5')
            print(buy_transaction_df)

            if buy_transaction_df.empty:
                print('No more buy orders')

            else: #try executing mid spread
                for index,row in buy_transaction_df.iterrows():
                    ticker = row['Ticker']
                    limit_price = round((row['ask'] + row['bid'])/2,2)
                    quantity = row['shares_to_trade_rounded']
                    orders.preview_and_place_limit_order("GOOD_FOR_DAY",ticker,'BUY',limit_price,quantity)


                #wait 30 seconds, cancel orders, check portfolio and calc trades
                time.sleep(30) #give 30 seconds for orders to execute
                orders.cancel_orders(account_ID_key) #cancels all outstanding orders
                time.sleep(5) #wait 5 seconds for cancel orders to run

                #check if trade orders executed by comparing portfolio to previous "current_portfolio_df"
                portfolio_after_trade_df = accounts.portfolio(account_ID_key) #recalc current portfolio
                before_after_portfolio_df = pd.merge(current_portfolio_df, portfolio_after_trade_df, on = 'Ticker', how = 'outer', suffixes = ('_prior','_after'))
                before_after_portfolio_df['Quantity_prior'] = before_after_portfolio_df['Quantity_prior'].fillna(0) #will have na if no position before trade, enter as 0
                before_after_portfolio_df['Quantity_after'] = before_after_portfolio_df['Quantity_after'].fillna(0) #will have na if no position after trade, enter as 0                                     
                before_after_portfolio_df['Trades'] = before_after_portfolio_df['Quantity_prior'] -  before_after_portfolio_df['Quantity_after']

                merged_df = pd.merge(transaction_df, before_after_portfolio_df, on='Ticker', how = 'left') #merge with transaction df, keeping transaction_df rows only
                merged_df['Trades'] = merged_df['Trades'].fillna(0) #if value in transaction_df was not in before_after_portfolio_df, then value in 'Trades' = nan, need to replace that with 0 b/c it was not traded
                merged_df = merged_df[~merged_df['Trades'].ne(0)] #remove rows where trades <>0 that identifies that a trade went through #double negative ~ (not) & ne (not equal)
                transaction_df = merged_df
                transaction_df = transaction_df[['Ticker','Quantity','weight','last_trade','bid','ask','order','shares_to_trade_rounded','transaction_value_rounded']]
                buy_transaction_df = transaction_df[transaction_df['order'].str.contains('Buy',case=False)]
                print('buy transaction df 2')
                print(buy_transaction_df)

                if buy_transaction_df.empty:
                    print('No more buy orders')

                else: #try executing at ask (1st time)
                    for index,row in buy_transaction_df.iterrows():
                        ticker = row['Ticker']
                        limit_price = row['ask']
                        quantity = row['shares_to_trade_rounded']
                        orders.preview_and_place_limit_order("GOOD_FOR_DAY",ticker,'BUY',limit_price,quantity)

                    #wait 30 seconds, cancel orders, check portfolio and calc trades
                    time.sleep(30) #give 30 seconds for orders to execute
                    orders.cancel_orders(account_ID_key) #cancels all outstanding orders
                    time.sleep(5) #wait 5 seconds for cancel orders to run

                    #check if trade orders executed by comparing portfolio to previous "current_portfolio_df"
                    portfolio_after_trade_df = accounts.portfolio(account_ID_key) #recalc current portfolio
                    before_after_portfolio_df = pd.merge(current_portfolio_df, portfolio_after_trade_df, on = 'Ticker', how = 'outer', suffixes = ('_prior','_after'))
                    before_after_portfolio_df['Quantity_prior'] = before_after_portfolio_df['Quantity_prior'].fillna(0) #will have na if no position before trade, enter as 0
                    before_after_portfolio_df['Quantity_after'] = before_after_portfolio_df['Quantity_after'].fillna(0) #will have na if no position after trade, enter as 0                                     
                    before_after_portfolio_df['Trades'] = before_after_portfolio_df['Quantity_prior'] -  before_after_portfolio_df['Quantity_after']

                    merged_df = pd.merge(transaction_df, before_after_portfolio_df, on='Ticker', how = 'left') #merge with transaction df, keeping transaction_df rows only
                    merged_df['Trades'] = merged_df['Trades'].fillna(0) #if value in transaction_df was not in before_after_portfolio_df, then value in 'Trades' = nan, need to replace that with 0 b/c it was not traded
                    merged_df = merged_df[~merged_df['Trades'].ne(0)] #remove rows where trades <>0 that identifies that a trade went through #double negative ~ (not) & ne (not equal)
                    transaction_df = merged_df
                    transaction_df = transaction_df[['Ticker','Quantity','weight','last_trade','bid','ask','order','shares_to_trade_rounded','transaction_value_rounded']]
                    buy_transaction_df = transaction_df[transaction_df['order'].str.contains('Buy',case=False)]
                    print('buy transaction df 3')
                    print(buy_transaction_df)

                    if buy_transaction_df.empty:
                        print('No more buy orders')

                    else: #try executing at ask (2nd time)
                        for index,row in buy_transaction_df.iterrows():
                            ticker = row['Ticker']
                            limit_price = row['ask']
                            quantity = row['shares_to_trade_rounded']
                            orders.preview_and_place_limit_order("GOOD_FOR_DAY",ticker,'BUY',limit_price,quantity)

                        #wait 30 seconds, cancel orders, check portfolio and calc trades
                        time.sleep(30) #give 30 seconds for orders to execute
                        orders.cancel_orders(account_ID_key) #cancels all outstanding orders
                        time.sleep(5) #wait 5 seconds for cancel orders to run

                        #check if trade orders executed by comparing portfolio to previous "current_portfolio_df"
                        portfolio_after_trade_df = accounts.portfolio(account_ID_key) #recalc current portfolio
                        before_after_portfolio_df = pd.merge(current_portfolio_df, portfolio_after_trade_df, on = 'Ticker', how = 'outer', suffixes = ('_prior','_after'))
                        before_after_portfolio_df['Quantity_prior'] = before_after_portfolio_df['Quantity_prior'].fillna(0) #will have na if no position before trade, enter as 0
                        before_after_portfolio_df['Quantity_after'] = before_after_portfolio_df['Quantity_after'].fillna(0) #will have na if no position after trade, enter as 0                                     
                        before_after_portfolio_df['Trades'] = before_after_portfolio_df['Quantity_prior'] -  before_after_portfolio_df['Quantity_after']

                        merged_df = pd.merge(transaction_df, before_after_portfolio_df, on='Ticker', how = 'left') #merge with transaction df, keeping transaction_df rows only
                        merged_df['Trades'] = merged_df['Trades'].fillna(0) #if value in transaction_df was not in before_after_portfolio_df, then value in 'Trades' = nan, need to replace that with 0 b/c it was not traded
                        merged_df = merged_df[~merged_df['Trades'].ne(0)] #remove rows where trades <>0 that identifies that a trade went through #double negative ~ (not) & ne (not equal)
                        transaction_df = merged_df
                        transaction_df = transaction_df[['Ticker','Quantity','weight','last_trade','bid','ask','order','shares_to_trade_rounded','transaction_value_rounded']]
                        buy_transaction_df = transaction_df[transaction_df['order'].str.contains('Buy',case=False)]
                        print('buy transaction df 4')
                        print(buy_transaction_df)

                        if buy_transaction_df.empty:
                            print('No more buy orders')

                        else: #try executing at ask (3rd time)
                            for index,row in buy_transaction_df.iterrows():
                                ticker = row['Ticker']
                                limit_price = row['ask']
                                quantity = row['shares_to_trade_rounded']
                                orders.preview_and_place_limit_order("GOOD_FOR_DAY",ticker,'BUY',limit_price,quantity)


                            #wait 30 seconds, cancel orders, check portfolio and calc trades
                            time.sleep(30) #give 30 seconds for orders to execute
                            orders.cancel_orders(account_ID_key) #cancels all outstanding orders
                            time.sleep(5) #wait 5 seconds for cancel orders to run

                            #check if trade orders executed by comparing portfolio to previous "current_portfolio_df"
                            portfolio_after_trade_df = accounts.portfolio(account_ID_key) #recalc current portfolio
                            before_after_portfolio_df = pd.merge(current_portfolio_df, portfolio_after_trade_df, on = 'Ticker', how = 'outer', suffixes = ('_prior','_after'))
                            before_after_portfolio_df['Quantity_prior'] = before_after_portfolio_df['Quantity_prior'].fillna(0) #will have na if no position before trade, enter as 0
                            before_after_portfolio_df['Quantity_after'] = before_after_portfolio_df['Quantity_after'].fillna(0) #will have na if no position after trade, enter as 0                                     
                            before_after_portfolio_df['Trades'] = before_after_portfolio_df['Quantity_prior'] -  before_after_portfolio_df['Quantity_after']

                            merged_df = pd.merge(transaction_df, before_after_portfolio_df, on='Ticker', how = 'left') #merge with transaction df, keeping transaction_df rows only
                            merged_df['Trades'] = merged_df['Trades'].fillna(0) #if value in transaction_df was not in before_after_portfolio_df, then value in 'Trades' = nan, need to replace that with 0 b/c it was not traded
                            merged_df = merged_df[~merged_df['Trades'].ne(0)] #remove rows where trades <>0 that identifies that a trade went through #double negative ~ (not) & ne (not equal)
                            transaction_df = merged_df
                            transaction_df = transaction_df[['Ticker','Quantity','weight','last_trade','bid','ask','order','shares_to_trade_rounded','transaction_value_rounded']]
                            buy_transaction_df = transaction_df[transaction_df['order'].str.contains('Buy',case=False)]
                            print('buy transaction df 5')
                            print(buy_transaction_df)
                            
                            if buy_transaction_df.empty:
                                print('No more buy orders')

                            else: #try executing at market
                                for index,row in buy_transaction_df.iterrows():
                                    ticker = row['Ticker']
                                    quantity = row['shares_to_trade_rounded']
                                    orders.preview_and_place_market_order(ticker,'BUY',quantity)

                                time.sleep(300) #give 300 seconds for orders to execute
                                orders.cancel_orders(account_ID_key) #cancels all outstanding orders
                                time.sleep(5) #wait 5 seconds for cancel orders to run

                                #check if trade orders executed by comparing portfolio to previous "current_portfolio_df"
                                portfolio_after_trade_df = accounts.portfolio(account_ID_key) #recalc current portfolio
                                before_after_portfolio_df = pd.merge(current_portfolio_df, portfolio_after_trade_df, on = 'Ticker', how = 'outer', suffixes = ('_prior','_after'))
                                before_after_portfolio_df['Quantity_prior'] = before_after_portfolio_df['Quantity_prior'].fillna(0) #will have na if no position before trade, enter as 0
                                before_after_portfolio_df['Quantity_after'] = before_after_portfolio_df['Quantity_after'].fillna(0) #will have na if no position after trade, enter as 0                                     
                                before_after_portfolio_df['Trades'] = before_after_portfolio_df['Quantity_prior'] -  before_after_portfolio_df['Quantity_after']

                                merged_df = pd.merge(transaction_df, before_after_portfolio_df, on='Ticker', how = 'left') #merge with transaction df, keeping transaction_df rows only
                                merged_df['Trades'] = merged_df['Trades'].fillna(0) #if value in transaction_df was not in before_after_portfolio_df, then value in 'Trades' = nan, need to replace that with 0 b/c it was not traded
                                merged_df = merged_df[~merged_df['Trades'].ne(0)] #remove rows where trades <>0 that identifies that a trade went through #double negative ~ (not) & ne (not equal)
                                transaction_df = merged_df
                                transaction_df = transaction_df[['Ticker','Quantity','weight','last_trade','bid','ask','order','shares_to_trade_rounded','transaction_value_rounded']]
                                buy_transaction_df = transaction_df[transaction_df['order'].str.contains('Buy',case=False)]
                                print('buy transaction df 6')
                                print(buy_transaction_df)

                                if buy_transaction_df.empty:
                                    orders.cancel_orders(account_ID_key) #cancels all outstanding orders                        
                                    print('No more buy orders')

                                else:
                                    orders.cancel_orders(account_ID_key) #cancels all outstanding orders 
                                    print("ORDERS UNABLE TO EXECUTE:")
                                    print(buy_transaction_df)
                                    
except:
    error = pd.DataFrame({'date':[end_date.strftime('%Y-%m-%d')],
                        'error': ['Error in buy order execution']})
    error.set_index('date',inplace=True)
    error_combined = pd.concat([error_df,error], ignore_index = False)
    
    writer = pd.ExcelWriter(error_file)
    error_combined.to_excel(writer, index=True)
    writer.close()
    raise ValueError('Error')

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 74, in emit
    self.doRollover()
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 179, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 115, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log' -> 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log.1'
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\reedx\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\reedx\anaconda3\Lib\site-packages\traitlets\config\application.


Open Orders: 
None
buy transaction df 1
  Ticker Quantity  weight  last_trade     bid     ask order  \
0   IBIT      NaN     0.1       58.79   59.99   60.00   Buy   
1   BITQ      NaN     0.1       19.73   19.73   22.79   Buy   
2   ARKK      NaN     0.1       62.69   62.96   63.00   Buy   
3   ARKX      NaN     0.1       21.39   20.53   21.80   Buy   
4   JETS      NaN     0.1       26.27   23.69   26.71   Buy   
5   IBLC      NaN     0.1       38.86   30.46   46.58   Buy   
6    EIS      NaN     0.1       81.38   72.47   93.14   Buy   
7    XTL      NaN     0.1      112.07   96.65  128.57   Buy   
8    XLY      NaN     0.1      231.42  231.02  231.96   Buy   
9   CLOU      NaN     0.1       24.85   25.01   25.19   Buy   

   shares_to_trade_rounded  transaction_value_rounded  
0                       18                    1058.22  
1                       55                    1085.15  
2                       17                    1065.73  
3                       51               

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 74, in emit
    self.doRollover()
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 179, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 115, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log' -> 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log.1'
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\reedx\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\reedx\anaconda3\Lib\site-packages\traitlets\config\application.

Error: We cannot accept this order because there are insufficient funds in your account.


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 74, in emit
    self.doRollover()
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 179, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 115, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log' -> 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log.1'
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\reedx\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\reedx\anaconda3\Lib\site-packages\traitlets\config\application.

...

Error: This order cannot be accepted due to insufficient funds in your account.The order exceeds your available purchasing power by approximately $1064.77. At the time of order preview, the maximum allowable quantity was estimated to be 1.


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 74, in emit
    self.doRollover()
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 179, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 115, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log' -> 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log.1'
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\reedx\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\reedx\anaconda3\Lib\site-packages\traitlets\config\application.


Open Orders: 
None


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 74, in emit
    self.doRollover()
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 179, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\Users\reedx\anaconda3\Lib\logging\handlers.py", line 115, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log' -> 'C:\\Users\\reedx\\OneDrive\\Investing\\Python\\Code\\Code packages\\python_client.log.1'
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\reedx\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\reedx\anaconda3\Lib\site-packages\traitlets\config\application.

buy transaction df 6
  Ticker Quantity  weight  last_trade     bid     ask order  \
0   IBIT      NaN     0.1       58.79   59.99   60.00   Buy   
1   BITQ      NaN     0.1       19.73   19.73   22.79   Buy   
2   ARKK      NaN     0.1       62.69   62.96   63.00   Buy   
3   ARKX      NaN     0.1       21.39   20.53   21.80   Buy   
4   JETS      NaN     0.1       26.27   23.69   26.71   Buy   
5   IBLC      NaN     0.1       38.86   30.46   46.58   Buy   
6    EIS      NaN     0.1       81.38   72.47   93.14   Buy   
7    XTL      NaN     0.1      112.07   96.65  128.57   Buy   
8    XLY      NaN     0.1      231.42  231.02  231.96   Buy   
9   CLOU      NaN     0.1       24.85   25.01   25.19   Buy   

   shares_to_trade_rounded  transaction_value_rounded  
0                       18                    1058.22  
1                       55                    1085.15  
2                       17                    1065.73  
3                       51                    1090.89  
4    